In [ ]:
%load_ext autoreload
%autoreload 2

## Notebook pour tester le pipe de fleet-telemetry

### Raw tss

In [ ]:
from core.s3_utils import *
from transform.raw_tss.fleet_telemetry_raw_tss import *
import pandas as pd
import plotly.express as px
from transform.processed_tss.config import *
from core.constants import *
from transform.raw_tss.config import *
from transform.processed_tss.ProcessedTimeSeries import ProcessedTimeSeries
from transform.raw_results.tesla_fleet_telemetry import get_results as get_results_origin
from transform.processed_results.main import get_processed_results
from datetime import timedelta

import numpy as np

In [ ]:
S3_RAW_TSS_KEY_FORMAT = "raw_ts/{brand}/time_series/raw_tss.parquet"
FLEET_TELEMETRY_RAW_TSS_KEY = S3_RAW_TSS_KEY_FORMAT.format(brand="tesla-fleet-telemetry")

In [ ]:
FLEET_TELEMETRY_RAW_TSS_KEY

In [ ]:
s3 = S3_Bucket()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max as spark_max, substring
from typing import List
from pyspark.sql import SparkSession, DataFrame as SparkDF
from pyspark.sql.functions import col, pandas_udf
from typing import Iterator
import pandas as pd

def get_response_keys_to_parse_with_spark(bucket: S3_Bucket, spark: SparkSession) -> "pyspark.sql.DataFrame":
    if bucket.check_file_exists(FLEET_TELEMETRY_RAW_TSS_KEY):
        # raw_tss_path = bucket.get_s3_path(FLEET_TELEMETRY_RAW_TSS_KEY)
        raw_tss_df = spark.read.parquet(FLEET_TELEMETRY_RAW_TSS_KEY).select("vin", "readable_date")
    else:
        # Attention : ici, DEFAULT_TESLA_RAW_TSS_DF doit être converti en PySpark
        raw_tss_df = spark.createDataFrame(DEFAULT_TESLA_RAW_TSS_DF)

    # Obtenir la dernière date parsée par VIN
    last_parsed_df = raw_tss_df.groupBy("vin").agg(
        spark_max("readable_date").alias("last_parsed_date")
    )

    # Liste des fichiers à parser
    responses_df = bucket.list_responses_keys_of_brand("tesla-fleet-telemetry", as_spark=True, spark=spark)

    # Extraire la date depuis le nom de fichier (supposé être à la fin du nom sans extension .jsonl ou .parquet)
    responses_df = responses_df.withColumn(
        "date", col("file").substr(1, length=col("file"))[:-5].cast("timestamp")
    )

    # Joindre avec les dernières dates
    merged_df = responses_df.join(last_parsed_df, on="vin", how="outer")

    # Garder les fichiers nouveaux ou non parsés
    filtered_df = merged_df.filter(
        (col("last_parsed_date").isNull()) | (col("date") > col("last_parsed_date"))
    )

    return filtered_df


def get_raw_tss_from_keys_with_spark(keys: SparkDF, bucket: S3_Bucket, spark: SparkSession, batch_size: int = 500) -> SparkDF:
    # Trier par date pour reproductibilité
    keys = keys.orderBy("date")

    # Conversion en partitions logiques avec repartition
    num_partitions = max(1, keys.count() // batch_size)
    keys = keys.repartition(num_partitions)

    # Définir une fonction de traitement par partition (pandas_udf de type GROUPED_MAP)
    @pandas_udf("vin string, timestamp timestamp, other_column_1 type, ...", functionType="MAP_ITER")
    def parse_partition(batch_iter: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
        for pdf in batch_iter:
            parsed_dfs = []
            for _, row in pdf.iterrows():
                key = row["key"]
                try:
                    response = bucket.read_json(key)
                    parsed_df = explode_data(pd.DataFrame.from_records(response))
                    parsed_dfs.append(parsed_df)
                except Exception as e:
                    logger.warning(f"Failed to parse {key}: {e}")
            if parsed_dfs:
                yield pd.concat(parsed_dfs, ignore_index=True)

    # Appliquer la fonction sur les partitions
    raw_tss_df = keys.mapInPandas(parse_partition, schema="vin string, timestamp timestamp, ...")

    return raw_tss_df

def get_raw_tss_with_spark(bucket: S3_Bucket, spark: SparkSession) -> "pyspark.sql.DataFrame":
    logger.debug("Getting raw TSS from responses provided by Tesla fleet telemetry.")
    keys_df = get_response_keys_to_parse_with_spark(bucket, spark)

    if bucket.check_file_exists(FLEET_TELEMETRY_RAW_TSS_KEY):
        # raw_tss_path = bucket.get_s3_path(FLEET_TELEMETRY_RAW_TSS_KEY)
        raw_tss_df = spark.read.parquet("s3://"+FLEET_TELEMETRY_RAW_TSS_KEY)
        new_raw_tss_df = get_raw_tss_from_keys_with_spark(keys_df, bucket, spark)
        return raw_tss_df.unionByName(new_raw_tss_df)
    else:
        return get_raw_tss_from_keys_with_spark(keys_df, bucket, spark)


In [ ]:
S3_Bucket().get_creds_from_dot_env()#.read_json_file("response/tesla-fleet-telemetry/7SAXCCE67RF445292/2025-04-02.json")

In [ ]:
access_key = "SCW9P6Q1T26F2JGSC1AS"
secret_key = "c702e16a-5a48-45f3-8538-5783b5c58e44"

In [ ]:
from pyspark.sql import SparkSession
import os
from transform.raw_tss.fleet_telemetry_raw_tss import *
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages org.apache.hadoop:hadoop-aws:3.3.4 pyspark-shell"

)
spark = SparkSession.builder \
    .appName("Scaleway S3 Read JSON") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.endpoint", "https://s3.fr-par.scw.cloud") \
    .config("spark.hadoop.fs.s3a.access.key", access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", secret_key) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .getOrCreate()

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType , MapType
from pyspark.sql.functions import col, max as spark_max, to_timestamp, expr, udf,  explode, when
from pyspark.sql import DataFrame
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, BooleanType, IntegerType, ArrayType


In [ ]:
def read_parquet_spark(key: str, columns: list[str] | None = None):
        full_path = f"s3a://bib-platform-prod-data/{key}"
        df = spark.read.parquet(full_path)
        if columns is not None:
            df = df.select(*columns)
        return df
    
def get_response_keys_to_parse_spark(bucket:S3_Bucket) -> DF:
    if bucket.check_file_exists(FLEET_TELEMETRY_RAW_TSS_KEY):
        raw_tss_subset = read_parquet_spark(FLEET_TELEMETRY_RAW_TSS_KEY, columns=["vin", "readable_date"])
    else:
        schema = StructType([
    StructField("vin", StringType(), True),
    StructField("readable_date", TimestampType(), True),
])
        raw_tss_subset = spark.createDataFrame([], schema)
    
    last_parsed_date = (
        raw_tss_subset
        .groupby(["vin"])
        .agg({"readable_date": "max"}).withColumnRenamed("max(readable_date)", "last_parsed_date")
    )
    response_keys_df = bucket.list_responses_keys_of_brand("tesla-fleet-telemetry")
    response_keys_df = spark.createDataFrame(response_keys_df)
    response_keys_df = response_keys_df.withColumn(
        "date",
        to_timestamp(expr("substring(file, 1, length(file) - 5)"))
    )

    return (
        response_keys_df
        .join(last_parsed_date, on="vin", how="outer")
        .filter((col("last_parsed_date").isNull()) | (col("date") > col("last_parsed_date")))
    )

In [ ]:
path = f"s3a://bib-platform-prod-data/response/tesla-fleet-telemetry/7SAXCCE67RF445292/2025-04-02.json"
path_parquet = "s3a://bib-platform-prod-data/raw_ts/tesla-fleet-telemetry/time_series/raw_tss.parquet"
df = spark.read.parquet(path_parquet)


In [ ]:
response = get_response_keys_to_parse_spark(s3)

In [ ]:
def flatten_metrics_udf(metrics_list):
    if not isinstance(metrics_list, list):
        return {}
    flat = {}
    for item in metrics_list:
        print('ici')
        key = item.get('key')
        value_dict = item.get('value', {})
        for subkey, subval in value_dict.items():
            flat[f"{key}_{subkey}"] = str(subval) if subval is not None else None
    return flat

flatten_metrics = udf(flatten_metrics_udf, MapType(StringType(), StringType()))

def explode_data_spark(df: DataFrame) -> DataFrame:
    """
    Attend un Spark DataFrame avec colonnes : vin, timestamp, readable_date, data
    - 'data' doit contenir une colonne de liste de métriques (JSON)
    - Retourne un DF flattené avec les colonnes dynamiques ajoutées
    """
    if "data" not in df.columns:
        return df

    # Appliquer le flatten avec la UDF
    df = df.withColumn("flat_data", flatten_metrics(col("data")))
    print(df)
    # Extraire dynamiquement les clés de flat_data (colonnes dynamiques)
    keys = df.selectExpr("explode(map_keys(flat_data)) as key").distinct().rdd.map(lambda r: r["key"]).collect()

    # Générer les colonnes dynamiques
    for k in keys:
        df = df.withColumn(k, col("flat_data").getItem(k))

    # Sélectionner les colonnes finales
    fixed_columns = ["vin", "timestamp", "readable_date"]
    return df.select(*fixed_columns, *keys)

In [ ]:
def explode_data_spark(response):
    value_schema = StructType([
    StructField("doubleValue", DoubleType(), True),
    StructField("stringValue", StringType(), True),
    StructField("booleanValue", BooleanType(), True),
    StructField("intValue", StringType(), True),
    StructField("climateKeeperModeValue", StringType(), True),
    StructField("hvacAutoModeValue", StringType(), True),
    StructField("hvacPowerValue", StringType(), True),
    StructField("detailedChargeStateValue", StringType(), True),
    StructField("defrostModeValue", StringType(), True),
    StructField("cableTypeValue", StringType(), True),
])

    # Struct de chaque élément de ce qui est contenu dans la colonne 'data'
    data_struct = StructType([
        StructField("key", StringType(), True),
        StructField("value", value_schema, True),
    ])

    # Schéma complet du DataFrame
    full_schema = StructType([
        StructField("vin", StringType(), True),
        StructField("timestamp", StringType(), True),
        StructField("readable_date", StringType(), True),
        StructField("data", ArrayType(data_struct), True),
    ])

    # Transfo le schéma
    df_raw = spark.read.schema(full_schema).json(spark.sparkContext.parallelize(response))
    print(df_raw.columns)
    #df_exploded = df_raw.withColumn("data_exploded", explode("data"))
    df_exploded = df_raw.withColumn("data_exploded", explode("data"))

    value_col = col("data_exploded.value")

    # Définir les métriques ! bug avec les boolean ! pas la solution pour le moment mais on s'en fou des colonnes donc pas grave 
    metric_value_expr = (
        when(value_col.getField("stringValue").isNotNull(), value_col.getField("stringValue"))
        .when(value_col.getField("doubleValue").isNotNull(), value_col.getField("doubleValue"))
        .when(value_col.getField("intValue").isNotNull(), value_col.getField("intValue"))
        #.when(value_col.getField("booleanValue").isNotNull(), value_col.getField("booleanValue"))
        .when(value_col.getField("detailedChargeStateValue").isNotNull(), value_col.getField("detailedChargeStateValue"))
        .when(value_col.getField("climateKeeperModeValue").isNotNull(), value_col.getField("climateKeeperModeValue"))
        .when(value_col.getField("hvacAutoModeValue").isNotNull(), value_col.getField("hvacAutoModeValue"))
        .when(value_col.getField("hvacPowerValue").isNotNull(), value_col.getField("hvacPowerValue"))
        .when(value_col.getField("defrostModeValue").isNotNull(), value_col.getField("defrostModeValue"))
        .when(value_col.getField("cableTypeValue").isNotNull(), value_col.getField("cableTypeValue"))
        .otherwise(None)
    )


    df_flat = df_exploded.select(
        col("vin"),
        col("timestamp"),
        col("readable_date"),
        col("data_exploded.key").alias("metric_key"),
        metric_value_expr.alias("metric_value")
    )

    # forcer l'affichage des colonnes à voir si on peut le supprimer à termes mais pas sur. 
    expected_keys = [
        'CarType', 'Odometer', 'DCChargingPower', 'BatteryHeaterOn', 'DetailedChargeState',
        'ClimateKeeperMode', 'HvacAutoMode', 'FastChargerPresent', 'DCDCEnable',
        'ChargeEnableRequest', 'ChargeCurrentRequest', 'ChargeCurrentRequestMax', 'ChargePort',
        'ModuleTempMin', 'EfficiencyPackage', 'InsideTemp', 'OutsideTemp', 'IsolationResistance',
        'ChargePortColdWeatherMode', 'SentryMode', 'ChargeLimitSoc', 'BrickVoltageMin',
        'BrickVoltageMax', 'EstBatteryRange', 'HvacPower', 'RatedRange', 'HvacFanSpeed',
        'DefrostMode', 'PreconditioningEnabled', "BatteryLevel", "PackCurrent", "LifetimeEnergyUsed", "DCChargingEnergyIn",
        "ChargerVoltage","EnergyRemaining","Soc","PackVoltage","IdealBatteryRange","ModuleTempMax","BMSState",
        "ChargeRateMilePerHour","ACChargingEnergyIn","ACChargingPower","BmsFullchargecomplete","ChargeState","ChargeAmps"
        ]

    #Groupby et pivot des métriques
    df_grouped = df_flat.groupBy("vin", "timestamp", "readable_date") \
        .pivot("metric_key", expected_keys) \
        .agg(expr("first(metric_value)"))

    return df_grouped

    df_grouped.show(truncate=False)

In [ ]:
def get_raw_tss_from_keys_spark(keys: DataFrame, bucket: S3_Bucket) -> DataFrame:
    """
    keys: Spark DataFrame with columns 'key', 'vin', 'date' (where 'date' is a timestamp)
    bucket: S3_Bucket with method read_multiple_json_files(keys: List[str], ...) -> List[dict]
    Returns a Spark DataFrame with parsed and exploded telemetry data
    """
    # Convert Spark DataFrame to Pandas for grouping
    keys_pd = keys.select("key", "vin", "date").toPandas()

    # Grouper par semaine (comme avant)
    grouped = keys_pd.groupby(pd.Grouper(key="date", freq="W-MON"))
    grouped_items = list(grouped)
    for week, week_keys in track(grouped_items, description="Processing weekly groups"):
        week_date = week.date().strftime('%Y-%m-%d')
        logger.debug(f"Parsing the responses of the week {week_date}:")
        logger.debug(f"{len(week_keys)} keys to parse for {week_keys['vin'].nunique()} vins.")
        logger.debug(f"This represents {round(len(week_keys) / len(keys_pd) * 100)}% of the total keys to parse.")

        # Lire les JSON depuis S3
        responses = bucket.read_multiple_json_files(week_keys["key"].tolist(), max_workers=64)
        print(len(responses))
        logger.debug(f"Read the responses.")
        # Parser et exploser les JSON avec ThreadPoolExecutor
        with ThreadPoolExecutor(max_workers=64) as executor:
            week_raw_tss = list(executor.map(explode_data_spark, responses)) # retourne des pandas ou Spark DFs ?

    print(len(week_raw_tss))
    # Union finale
    from functools import reduce
    from pyspark.sql import DataFrame
    if week_raw_tss:
        return reduce(DataFrame.unionAll, week_raw_tss)
    else:
        # Retourne un Spark DF vide si aucun résultat
        schema = spark.createDataFrame([], week_raw_tss[0].schema) if 'concatenated' in locals() else spark.createDataFrame([], schema=StructType([]))
        return schema

In [ ]:
line_test = response.head(7)

In [ ]:
test_response = [{'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747835682474, 'readable_date': '2025-05-21 13:54:42', 'data': [{'key': 
'PackVoltage', 'value': {'stringValue': '373.6499916482717'}}, {'key': 'ACChargingPower', 'value': {'stringValue': 
'0'}}, {'key': 'LifetimeEnergyUsed', 'value': {'stringValue': '26561.601261608303'}}, {'key': 'CarType', 'value': 
{'stringValue': 'Model3'}}, {'key': 'Odometer', 'value': {'stringValue': '63928.977202334645'}}, {'key': 
'DCChargingEnergyIn', 'value': {'stringValue': '17.339999612420797'}}, {'key': 'ChargerPhases', 'value': 
{'invalid': True}}, {'key': 'DCChargingPower', 'value': {'stringValue': '0'}}, {'key': 'BatteryHeaterOn', 'value': 
{'stringValue': 'false'}}, {'key': 'BmsFullchargecomplete', 'value': {'stringValue': 'true'}}, {'key': 
'DetailedChargeState', 'value': {'detailedChargeStateValue': 'DetailedChargeStateComplete'}}, {'key': 
'ClimateKeeperMode', 'value': {'climateKeeperModeValue': 'ClimateKeeperModeStateOff'}}, {'key': 'HvacAutoMode', 
'value': {'hvacAutoModeValue': 'HvacAutoModeStateOn'}}, {'key': 'FastChargerPresent', 'value': {'stringValue': 
'false'}}, {'key': 'ChargeAmps', 'value': {'stringValue': '0'}}, {'key': 'DCDCEnable', 'value': {'stringValue': 
'false'}}, {'key': 'ChargeEnableRequest', 'value': {'stringValue': 'true'}}, {'key': 'ChargeCurrentRequest', 
'value': {'stringValue': '16'}}, {'key': 'ChargeCurrentRequestMax', 'value': {'stringValue': '16'}}, {'key': 
'ChargeRateMilePerHour', 'value': {'doubleValue': 0}}, {'key': 'EstimatedHoursToChargeTermination', 'value': 
{'invalid': True}}, {'key': 'ChargingCableType', 'value': {'cableTypeValue': 'CableTypeIEC'}}, {'key': 
'ChargePort', 'value': {'stringValue': 'CCS'}}, {'key': 'ModuleTempMin', 'value': {'stringValue': '20'}}, {'key': 
'EfficiencyPackage', 'value': {'stringValue': 'Default'}}, {'key': 'InsideTemp', 'value': {'stringValue': 
'19.50000088661909'}}, {'key': 'OutsideTemp', 'value': {'stringValue': '19'}}, {'key': 'FastChargerType', 'value': 
{'invalid': True}}, {'key': 'IsolationResistance', 'value': {'stringValue': '4280'}}, {'key': 
'ChargePortColdWeatherMode', 'value': {'stringValue': 'false'}}, {'key': 'SentryMode', 'value': {'stringValue': 
'Off'}}, {'key': 'ChargeLimitSoc', 'value': {'stringValue': '60'}}, {'key': 'BMSState', 'value': {'stringValue': 
'Standby'}}, {'key': 'BrickVoltageMin', 'value': {'stringValue': '3.8940001849550754'}}, {'key': 
'DefrostForPreconditioning', 'value': {'booleanValue': False}}, {'key': 'EuropeVehicle', 'value': {'booleanValue': 
True}}, {'key': 'PackCurrent', 'value': {'stringValue': '0'}}, {'key': 'BrickVoltageMax', 'value': {'stringValue': 
'3.8960001850500703'}}, {'key': 'EstBatteryRange', 'value': {'stringValue': '138.46325600610206'}}, {'key': 
'VehicleSpeed', 'value': {'invalid': True}}, {'key': 'EnergyRemaining', 'value': {'stringValue': 
'39.559999115765095'}}, {'key': 'ChargeState', 'value': {'stringValue': 'Idle'}}, {'key': 'HvacPower', 'value': 
{'hvacPowerValue': 'HvacPowerStateOff'}}, {'key': 'ModuleTempMax', 'value': {'stringValue': '21'}}, {'key': 
'RatedRange', 'value': {'stringValue': '155.02040469828918'}}, {'key': 'HvacFanSpeed', 'value': {'intValue': 2}}, 
{'key': 'ACChargingEnergyIn', 'value': {'stringValue': '19.64266695936988'}}, {'key': 'BatteryLevel', 'value': 
{'stringValue': '60.56782334384858'}}, {'key': 'IdealBatteryRange', 'value': {'stringValue': 
'155.02040469828918'}}, {'key': 'Soc', 'value': {'stringValue': '59.90536277602524'}}, {'key': 'DefrostMode', 
'value': {'defrostModeValue': 'DefrostModeStateOff'}}, {'key': 'HvacACEnabled', 'value': {'booleanValue': False}}, 
{'key': 'RearDefrostEnabled', 'value': {'booleanValue': False}}, {'key': 'PreconditioningEnabled', 'value': 
{'stringValue': 'false'}}], 'meta': {}, 'createdAt': '2025-05-21T13:54:42.474997411Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747835683974, 'readable_date': '2025-05-21 13:54:43', 'data': [{'key': 
'ChargerVoltage', 'value': {'doubleValue': 1.683}}], 'meta': {}, 'createdAt': '2025-05-21T13:54:43.974458426Z'}, 
{'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747835712475, 'readable_date': '2025-05-21 13:55:12', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.6000000089406967'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.4199916534126'}}, {'key': 'LifetimeEnergyUsed', 'value': {'stringValue': '26561.603261608398'}}], 'meta': {}, 
'createdAt': '2025-05-21T13:55:12.475527838Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747835742474, 
'readable_date': '2025-05-21 13:55:42', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.5000000074505806'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.5699916500598'}}, {'key': 
'LifetimeEnergyUsed', 'value': {'stringValue': '26561.607261608588'}}], 'meta': {}, 'createdAt': 
'2025-05-21T13:55:42.474523709Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747835772475, 'readable_date': 
'2025-05-21 13:56:12', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 
'PackVoltage', 'value': {'stringValue': '373.6299916487187'}}], 'meta': {}, 'createdAt': 
'2025-05-21T13:56:12.475427434Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747835787975, 'readable_date': 
'2025-05-21 13:56:27', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': '26561.609261608683'}}], 
'meta': {}, 'createdAt': '2025-05-21T13:56:27.975171893Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747835802474, 'readable_date': '2025-05-21 13:56:42', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.4199916534126'}}], 'meta': {}, 
'createdAt': '2025-05-21T13:56:42.474945724Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747835832474, 
'readable_date': '2025-05-21 13:57:12', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.42999165318906'}}], 'meta': {}, 
'createdAt': '2025-05-21T13:57:12.474758798Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747835839975, 
'readable_date': '2025-05-21 13:57:19', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.611261608778'}}], 'meta': {}, 'createdAt': '2025-05-21T13:57:19.975298424Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747835849975, 'readable_date': '2025-05-21 13:57:29', 'data': [{'key': 'BatteryLevel', 'value': 
{'stringValue': '60.53627760252366'}}], 'meta': {}, 'createdAt': '2025-05-21T13:57:29.975134758Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747835862474, 'readable_date': '2025-05-21 13:57:42', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.38999165408313'}}], 'meta': {}, 'createdAt': '2025-05-21T13:57:42.474418586Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747835892475, 'readable_date': '2025-05-21 13:58:12', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.5799916498363'}}], 
'meta': {}, 'createdAt': '2025-05-21T13:58:12.475352983Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747835892975, 'readable_date': '2025-05-21 13:58:12', 'data': [{'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.613261608873'}}], 'meta': {}, 'createdAt': '2025-05-21T13:58:12.975257928Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747835904975, 'readable_date': '2025-05-21 13:58:24', 'data': [{'key': 
'EnergyRemaining', 'value': {'stringValue': '39.53999911621213'}}], 'meta': {}, 'createdAt': 
'2025-05-21T13:58:24.975429200Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747835922474, 'readable_date': 
'2025-05-21 13:58:42', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': '373.36999165453017'}}], 'meta': 
{}, 'createdAt': '2025-05-21T13:58:42.474908630Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747835930975, 
'readable_date': '2025-05-21 13:58:50', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.5000000074505806'}}], 'meta': {}, 'createdAt': '2025-05-21T13:58:50.975126321Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747835934974, 'readable_date': '2025-05-21 13:58:54', 'data': [{'key': 'EnergyRemaining', 'value': 
{'stringValue': '39.53999911621213'}}], 'meta': {}, 'createdAt': '2025-05-21T13:58:54.974469349Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747835946975, 'readable_date': '2025-05-21 13:59:06', 'data': [{'key': 
'LifetimeEnergyUsed', 'value': {'stringValue': '26561.615261608968'}}], 'meta': {}, 'createdAt': 
'2025-05-21T13:59:06.975314450Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747835951974, 'readable_date': 
'2025-05-21 13:59:11', 'data': [{'key': 'DCChargingEnergyIn', 'value': {'stringValue': '17.319999612867832'}}, 
{'key': 'IdealBatteryRange', 'value': {'stringValue': '154.9387720470526'}}, {'key': 'Soc', 'value': 
{'stringValue': '59.873817034700316'}}], 'meta': {}, 'createdAt': '2025-05-21T13:59:11.974874867Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747835952476, 'readable_date': '2025-05-21 13:59:12', 'data': [{'key': 
'PackVoltage', 'value': {'stringValue': '373.48999165184796'}}], 'meta': {}, 'createdAt': 
'2025-05-21T13:59:12.476194805Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747835960975, 'readable_date': 
'2025-05-21 13:59:20', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}], 'meta': 
{}, 'createdAt': '2025-05-21T13:59:20.975226700Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747835982474, 
'readable_date': '2025-05-21 13:59:42', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': 
'373.4799916520715'}}], 'meta': {}, 'createdAt': '2025-05-21T13:59:42.474574655Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747835990975, 'readable_date': '2025-05-21 13:59:50', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}], 'meta': {}, 'createdAt': '2025-05-21T13:59:50.975201736Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747836001975, 'readable_date': '2025-05-21 14:00:01', 'data': [{'key': 
'LifetimeEnergyUsed', 'value': {'stringValue': '26561.617261609063'}}], 'meta': {}, 'createdAt': 
'2025-05-21T14:00:01.975166681Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747836012474, 'readable_date': 
'2025-05-21 14:00:12', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': '373.5799916498363'}}], 'meta': {},
'createdAt': '2025-05-21T14:00:12.474727310Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747836020974, 
'readable_date': '2025-05-21 14:00:20', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}], 'meta': {}, 'createdAt': '2025-05-21T14:00:20.974903679Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747836042474, 'readable_date': '2025-05-21 14:00:42', 'data': [{'key': 'PackVoltage', 'value': 
{'stringValue': '373.36999165453017'}}], 'meta': {}, 'createdAt': '2025-05-21T14:00:42.474962399Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747836050975, 'readable_date': '2025-05-21 14:00:50', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}], 'meta': {}, 'createdAt': 
'2025-05-21T14:00:50.975267990Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747836056974, 'readable_date': 
'2025-05-21 14:00:56', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': '26561.621261609253'}}], 
'meta': {}, 'createdAt': '2025-05-21T14:00:56.974851266Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747836072474, 'readable_date': '2025-05-21 14:01:12', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': 
'373.59999164938927'}}], 'meta': {}, 'createdAt': '2025-05-21T14:01:12.474486290Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747836080975, 'readable_date': '2025-05-21 14:01:20', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}], 'meta': {}, 'createdAt': '2025-05-21T14:01:20.975071454Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747836102474, 'readable_date': '2025-05-21 14:01:42', 'data': [{'key': 
'PackVoltage', 'value': {'stringValue': '373.5799916498363'}}], 'meta': {}, 'createdAt': 
'2025-05-21T14:01:42.474557407Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747836110975, 'readable_date': 
'2025-05-21 14:01:50', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}], 'meta': 
{}, 'createdAt': '2025-05-21T14:01:50.975097888Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747836112974, 
'readable_date': '2025-05-21 14:01:52', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.623261609348'}}], 'meta': {}, 'createdAt': '2025-05-21T14:01:52.974497958Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747836132474, 'readable_date': '2025-05-21 14:02:12', 'data': [{'key': 'PackVoltage', 'value': 
{'stringValue': '373.3999916538596'}}], 'meta': {}, 'createdAt': '2025-05-21T14:02:12.474407851Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747836140974, 'readable_date': '2025-05-21 14:02:20', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}], 'meta': {}, 'createdAt': 
'2025-05-21T14:02:20.974471700Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747836162474, 'readable_date': 
'2025-05-21 14:02:42', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': '373.2999916560948'}}], 'meta': {},
'createdAt': '2025-05-21T14:02:42.474745442Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747836169974, 
'readable_date': '2025-05-21 14:02:49', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.625261609443'}}], 'meta': {}, 'createdAt': '2025-05-21T14:02:49.974820554Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747836170975, 'readable_date': '2025-05-21 14:02:50', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}], 'meta': {}, 'createdAt': '2025-05-21T14:02:50.975100721Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747836192474, 'readable_date': '2025-05-21 14:03:12', 'data': [{'key': 
'PackVoltage', 'value': {'stringValue': '373.59999164938927'}}], 'meta': {}, 'createdAt': 
'2025-05-21T14:03:12.474627378Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747836222474, 'readable_date': 
'2025-05-21 14:03:42', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': '373.43999165296555'}}], 'meta': 
{}, 'createdAt': '2025-05-21T14:03:42.474649689Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747836226974, 
'readable_date': '2025-05-21 14:03:46', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.627261609538'}}], 'meta': {}, 'createdAt': '2025-05-21T14:03:46.974966741Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747836230974, 'readable_date': '2025-05-21 14:03:50', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}], 'meta': {}, 'createdAt': '2025-05-21T14:03:50.974893118Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747836252474, 'readable_date': '2025-05-21 14:04:12', 'data': [{'key': 
'PackVoltage', 'value': {'stringValue': '373.4099916536361'}}], 'meta': {}, 'createdAt': 
'2025-05-21T14:04:12.474576629Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747836260974, 'readable_date': 
'2025-05-21 14:04:20', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}], 'meta': 
{}, 'createdAt': '2025-05-21T14:04:20.974964935Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747836282475, 
'readable_date': '2025-05-21 14:04:42', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': 
'373.36999165453017'}}, {'key': 'BMSState', 'value': {'stringValue': 'Support'}}, {'key': 'ModuleTempMax', 'value':
{'stringValue': '21.5'}}], 'meta': {}, 'createdAt': '2025-05-21T14:04:42.475471873Z'}, {'vin': '5YJ3E7EB1KF334219',
'timestamp': 1747836283975, 'readable_date': '2025-05-21 14:04:43', 'data': [{'key': 'LifetimeEnergyUsed', 'value':
{'stringValue': '26561.629261609633'}}, {'key': 'BatteryLevel', 'value': {'stringValue': '60.504731861198735'}}], 
'meta': {}, 'createdAt': '2025-05-21T14:04:43.975430161Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747836290974, 'readable_date': '2025-05-21 14:04:50', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}], 'meta': {}, 'createdAt': '2025-05-21T14:04:50.974463931Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747836312475, 'readable_date': '2025-05-21 14:05:12', 'data': [{'key': 'PackVoltage', 'value': 
{'stringValue': '373.54999165050685'}}], 'meta': {}, 'createdAt': '2025-05-21T14:05:12.475443450Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747836320975, 'readable_date': '2025-05-21 14:05:20', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '0'}}], 'meta': {}, 'createdAt': '2025-05-21T14:05:20.975660319Z'}, {'vin':
'5YJ3E7EB1KF334219', 'timestamp': 1747839799616, 'readable_date': '2025-05-21 15:03:19', 'data': [{'key': 
'ModuleTempMax', 'value': {'stringValue': '20.5'}}, {'key': 'EuropeVehicle', 'value': {'booleanValue': True}}, 
{'key': 'ChargerPhases', 'value': {'invalid': True}}, {'key': 'ChargeEnableRequest', 'value': {'stringValue': 
'true'}}, {'key': 'RearDefrostEnabled', 'value': {'booleanValue': False}}, {'key': 'HvacFanSpeed', 'value': 
{'intValue': 2}}, {'key': 'DCChargingEnergyIn', 'value': {'stringValue': '17.319999612867832'}}, {'key': 
'EstimatedHoursToChargeTermination', 'value': {'invalid': True}}, {'key': 'Odometer', 'value': {'stringValue': 
'63928.977202334645'}}, {'key': 'DCChargingPower', 'value': {'stringValue': '0'}}, {'key': 'BMSState', 'value': 
{'stringValue': 'Standby'}}, {'key': 'HvacAutoMode', 'value': {'hvacAutoModeValue': 'HvacAutoModeStateOn'}}, 
{'key': 'PreconditioningEnabled', 'value': {'stringValue': 'false'}}, {'key': 'ACChargingPower', 'value': 
{'stringValue': '0'}}, {'key': 'EnergyRemaining', 'value': {'stringValue': '39.53999911621213'}}, {'key': 
'CarType', 'value': {'stringValue': 'Model3'}}, {'key': 'FastChargerType', 'value': {'invalid': True}}, {'key': 
'DCDCEnable', 'value': {'stringValue': 'false'}}, {'key': 'BatteryLevel', 'value': {'stringValue': 
'60.504731861198735'}}, {'key': 'EfficiencyPackage', 'value': {'stringValue': 'Default'}}, {'key': 'HvacACEnabled',
'value': {'booleanValue': False}}, {'key': 'ChargeRateMilePerHour', 'value': {'doubleValue': 0}}, {'key': 
'ChargePort', 'value': {'stringValue': 'CCS'}}, {'key': 'BmsFullchargecomplete', 'value': {'stringValue': 'true'}},
{'key': 'ACChargingEnergyIn', 'value': {'stringValue': '19.64266695936988'}}, {'key': 'ChargingCableType', 'value':
{'cableTypeValue': 'CableTypeIEC'}}, {'key': 'EstBatteryRange', 'value': {'stringValue': '138.3903422325338'}}, 
{'key': 'ChargeState', 'value': {'stringValue': 'Idle'}}, {'key': 'SentryMode', 'value': {'stringValue': 'Off'}}, 
{'key': 'ChargeLimitSoc', 'value': {'stringValue': '60'}}, {'key': 'FastChargerPresent', 'value': {'stringValue': 
'false'}}, {'key': 'DetailedChargeState', 'value': {'detailedChargeStateValue': 'DetailedChargeStateComplete'}}, 
{'key': 'DefrostMode', 'value': {'defrostModeValue': 'DefrostModeStateOff'}}, {'key': 'ChargeCurrentRequestMax', 
'value': {'stringValue': '16'}}, {'key': 'HvacPower', 'value': {'hvacPowerValue': 'HvacPowerStateOff'}}, {'key': 
'BrickVoltageMax', 'value': {'stringValue': '3.8960001850500703'}}, {'key': 'ChargeAmps', 'value': {'stringValue': 
'0'}}, {'key': 'VehicleSpeed', 'value': {'invalid': True}}, {'key': 'PackCurrent', 'value': {'stringValue': '0'}}, 
{'key': 'ModuleTempMin', 'value': {'stringValue': '20'}}, {'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.631261609728'}}, {'key': 'InsideTemp', 'value': {'stringValue': '19.400000885128975'}}, {'key': 
'OutsideTemp', 'value': {'stringValue': '19'}}, {'key': 'ClimateKeeperMode', 'value': {'climateKeeperModeValue': 
'ClimateKeeperModeStateOff'}}, {'key': 'IdealBatteryRange', 'value': {'stringValue': '154.9387720470526'}}, {'key':
'IsolationResistance', 'value': {'stringValue': '4280'}}, {'key': 'RatedRange', 'value': {'stringValue': 
'154.9387720470526'}}, {'key': 'ChargeCurrentRequest', 'value': {'stringValue': '16'}}, {'key': 
'DefrostForPreconditioning', 'value': {'booleanValue': False}}, {'key': 'BrickVoltageMin', 'value': {'stringValue':
'3.8920001848600805'}}, {'key': 'Soc', 'value': {'stringValue': '59.873817034700316'}}, {'key': 'BatteryHeaterOn', 
'value': {'stringValue': 'false'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.6799916476011'}}, {'key': 
'ChargePortColdWeatherMode', 'value': {'stringValue': 'false'}}], 'meta': {}, 'createdAt': 
'2025-05-21T15:03:19.616691793Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747839800618, 'readable_date': 
'2025-05-21 15:03:20', 'data': [{'key': 'ChargerVoltage', 'value': {'doubleValue': 1.716}}], 'meta': {}, 
'createdAt': '2025-05-21T15:03:20.618704348Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747839829617, 
'readable_date': '2025-05-21 15:03:49', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.9000000134110451'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.4199916534126'}}, {'key': 
'LifetimeEnergyUsed', 'value': {'stringValue': '26561.635261609918'}}], 'meta': {}, 'createdAt': 
'2025-05-21T15:03:49.617379944Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747839832117, 'readable_date': 
'2025-05-21 15:03:52', 'data': [{'key': 'Soc', 'value': {'stringValue': '59.84227129337539'}}, {'key': 
'DCChargingEnergyIn', 'value': {'stringValue': '17.299999613314867'}}], 'meta': {}, 'createdAt': 
'2025-05-21T15:03:52.117192586Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747839861442, 'readable_date': 
'2025-05-21 15:04:21', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 
'PackVoltage', 'value': {'stringValue': '373.6399916484952'}}, {'key': 'IdealBatteryRange', 'value': 
{'stringValue': '154.857139395816'}}], 'meta': {}, 'createdAt': '2025-05-21T15:04:21.442336304Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747839862942, 'readable_date': '2025-05-21 15:04:22', 'data': [{'key': 
'EnergyRemaining', 'value': {'stringValue': '39.559999115765095'}}], 'meta': {}, 'createdAt': 
'2025-05-21T15:04:22.942401018Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747839870942, 'readable_date': 
'2025-05-21 15:04:30', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': '26561.637261610013'}}], 
'meta': {}, 'createdAt': '2025-05-21T15:04:30.942457008Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747839891442, 'readable_date': '2025-05-21 15:04:51', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.5000000074505806'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.469991652295'}}], 'meta': {}, 
'createdAt': '2025-05-21T15:04:51.442805623Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747839921442, 
'readable_date': '2025-05-21 15:05:21', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.49999165162444'}}], 'meta': {}, 
'createdAt': '2025-05-21T15:05:21.442823420Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747839924942, 
'readable_date': '2025-05-21 15:05:24', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.639261610107'}}], 'meta': {}, 'createdAt': '2025-05-21T15:05:24.942599301Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747839951442, 'readable_date': '2025-05-21 15:05:51', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.42999165318906'}}], 
'meta': {}, 'createdAt': '2025-05-21T15:05:51.442789853Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747839979943, 'readable_date': '2025-05-21 15:06:19', 'data': [{'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.641261610202'}}], 'meta': {}, 'createdAt': '2025-05-21T15:06:19.943011216Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747839981442, 'readable_date': '2025-05-21 15:06:21', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.2999916560948'}}], 'meta': {}, 'createdAt': '2025-05-21T15:06:21.442118694Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747840011442, 'readable_date': '2025-05-21 15:06:51', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.54999165050685'}}], 
'meta': {}, 'createdAt': '2025-05-21T15:06:51.442707019Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747840036942, 'readable_date': '2025-05-21 15:07:16', 'data': [{'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.643261610297'}}], 'meta': {}, 'createdAt': '2025-05-21T15:07:16.942390044Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747840041442, 'readable_date': '2025-05-21 15:07:21', 'data': [{'key': 
'PackVoltage', 'value': {'stringValue': '373.36999165453017'}}, {'key': 'PackCurrent', 'value': {'stringValue': 
'-0.30000000447034836'}}], 'meta': {}, 'createdAt': '2025-05-21T15:07:21.442121514Z'}, {'vin': '5YJ3E7EB1KF334219',
'timestamp': 1747840071442, 'readable_date': '2025-05-21 15:07:51', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.3399916552007'}}], 
'meta': {}, 'createdAt': '2025-05-21T15:07:51.442266012Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747840092942, 'readable_date': '2025-05-21 15:08:12', 'data': [{'key': 'EnergyRemaining', 'value': {'stringValue':
'39.53999911621213'}}], 'meta': {}, 'createdAt': '2025-05-21T15:08:12.942791089Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747840094942, 'readable_date': '2025-05-21 15:08:14', 'data': [{'key': 'LifetimeEnergyUsed', 'value':
{'stringValue': '26561.645261610392'}}], 'meta': {}, 'createdAt': '2025-05-21T15:08:14.942867494Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747840101442, 'readable_date': '2025-05-21 15:08:21', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.5000000074505806'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.4599916525185'}}], 'meta': {}, 'createdAt': '2025-05-21T15:08:21.442235345Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747840131442, 'readable_date': '2025-05-21 15:08:51', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.30000000447034836'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.5199916511774'}}], 
'meta': {}, 'createdAt': '2025-05-21T15:08:51.442827305Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747840147942, 'readable_date': '2025-05-21 15:09:07', 'data': [{'key': 'BatteryLevel', 'value': {'stringValue': 
'60.47318611987381'}}], 'meta': {}, 'createdAt': '2025-05-21T15:09:07.942206768Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747840152942, 'readable_date': '2025-05-21 15:09:12', 'data': [{'key': 'LifetimeEnergyUsed', 'value':
{'stringValue': '26561.649261610582'}}], 'meta': {}, 'createdAt': '2025-05-21T15:09:12.942224608Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747840161442, 'readable_date': '2025-05-21 15:09:21', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.31999165564775'}}], 'meta': {}, 'createdAt': '2025-05-21T15:09:21.442705238Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747840191442, 'readable_date': '2025-05-21 15:09:51', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.30000000447034836'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.5199916511774'}}], 
'meta': {}, 'createdAt': '2025-05-21T15:09:51.442902400Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747840211942, 'readable_date': '2025-05-21 15:10:11', 'data': [{'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.651261610677'}}], 'meta': {}, 'createdAt': '2025-05-21T15:10:11.942212254Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747840221442, 'readable_date': '2025-05-21 15:10:21', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.31999165564775'}}], 'meta': {}, 'createdAt': '2025-05-21T15:10:21.442545161Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747840251442, 'readable_date': '2025-05-21 15:10:51', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.3599916547537'}}], 
'meta': {}, 'createdAt': '2025-05-21T15:10:51.442131993Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747840257942, 'readable_date': '2025-05-21 15:10:57', 'data': [{'key': 'IdealBatteryRange', 'value': 
{'stringValue': '154.77550674457942'}}, {'key': 'Soc', 'value': {'stringValue': '59.81072555205047'}}, {'key': 
'DCChargingEnergyIn', 'value': {'stringValue': '17.279999613761902'}}], 'meta': {}, 'createdAt': 
'2025-05-21T15:10:57.942729416Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747840270942, 'readable_date': 
'2025-05-21 15:11:10', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': '26561.653261610772'}}], 
'meta': {}, 'createdAt': '2025-05-21T15:11:10.942777590Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747840281442, 'readable_date': '2025-05-21 15:11:21', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.26999165676534'}}], 'meta': {}, 
'createdAt': '2025-05-21T15:11:21.442707192Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747840311442, 
'readable_date': '2025-05-21 15:11:51', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.30000000447034836'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.55999165028334'}}], 'meta': {}, 
'createdAt': '2025-05-21T15:11:51.442822041Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747840329942, 
'readable_date': '2025-05-21 15:12:09', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.655261610867'}}], 'meta': {}, 'createdAt': '2025-05-21T15:12:09.942141636Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747840341442, 'readable_date': '2025-05-21 15:12:21', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.36999165453017'}}], 
'meta': {}, 'createdAt': '2025-05-21T15:12:21.442783955Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747840371443, 'readable_date': '2025-05-21 15:12:51', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.30000000447034836'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.43999165296555'}}], 'meta': {}, 
'createdAt': '2025-05-21T15:12:51.443093096Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747840401442, 
'readable_date': '2025-05-21 15:13:21', 'data': [{'key': 'BMSState', 'value': {'stringValue': 'Support'}}, {'key': 
'ModuleTempMax', 'value': {'stringValue': '21'}}, {'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.179991658777'}}], 'meta': {}, 
'createdAt': '2025-05-21T15:13:21.442490914Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747840431442, 
'readable_date': '2025-05-21 15:13:51', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.30000000447034836'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.4199916534126'}}], 'meta': {}, 
'createdAt': '2025-05-21T15:13:51.442165954Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747847525612, 
'readable_date': '2025-05-21 17:12:05', 'data': [{'key': 'ChargingCableType', 'value': {'cableTypeValue': 
'CableTypeIEC'}}, {'key': 'EnergyRemaining', 'value': {'stringValue': '39.53999911621213'}}, {'key': 
'ChargeRateMilePerHour', 'value': {'doubleValue': 0}}, {'key': 'DetailedChargeState', 'value': 
{'detailedChargeStateValue': 'DetailedChargeStateComplete'}}, {'key': 'DefrostMode', 'value': {'defrostModeValue': 
'DefrostModeStateOff'}}, {'key': 'FastChargerType', 'value': {'invalid': True}}, {'key': 'EuropeVehicle', 'value': 
{'booleanValue': True}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.5899916496128'}}, {'key': 
'BrickVoltageMin', 'value': {'stringValue': '3.8920001848600805'}}, {'key': 'HvacFanSpeed', 'value': {'intValue': 
2}}, {'key': 'BMSState', 'value': {'stringValue': 'Standby'}}, {'key': 'EstBatteryRange', 'value': {'stringValue': 
'138.24451468539732'}}, {'key': 'ChargeState', 'value': {'stringValue': 'Idle'}}, {'key': 'ClimateKeeperMode', 
'value': {'climateKeeperModeValue': 'ClimateKeeperModeStateOff'}}, {'key': 'HvacACEnabled', 'value': 
{'booleanValue': False}}, {'key': 'ChargeAmps', 'value': {'stringValue': '0'}}, {'key': 'ACChargingPower', 'value':
{'stringValue': '0'}}, {'key': 'EfficiencyPackage', 'value': {'stringValue': 'Default'}}, {'key': 'OutsideTemp', 
'value': {'stringValue': '19'}}, {'key': 'FastChargerPresent', 'value': {'stringValue': 'false'}}, {'key': 
'BatteryLevel', 'value': {'stringValue': '60.47318611987381'}}, {'key': 'Soc', 'value': {'stringValue': 
'59.81072555205047'}}, {'key': 'ModuleTempMax', 'value': {'stringValue': '20.5'}}, {'key': 
'PreconditioningEnabled', 'value': {'stringValue': 'false'}}, {'key': 'ChargePortColdWeatherMode', 'value': 
{'stringValue': 'false'}}, {'key': 'EstimatedHoursToChargeTermination', 'value': {'invalid': True}}, {'key': 
'IdealBatteryRange', 'value': {'stringValue': '154.77550674457942'}}, {'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.657261610962'}}, {'key': 'Odometer', 'value': {'stringValue': '63928.977202334645'}}, 
{'key': 'ChargeCurrentRequestMax', 'value': {'stringValue': '16'}}, {'key': 'DCChargingPower', 'value': 
{'stringValue': '0'}}, {'key': 'ChargePort', 'value': {'stringValue': 'CCS'}}, {'key': 'ChargerPhases', 'value': 
{'invalid': True}}, {'key': 'HvacAutoMode', 'value': {'hvacAutoModeValue': 'HvacAutoModeStateOn'}}, {'key': 
'BmsFullchargecomplete', 'value': {'stringValue': 'true'}}, {'key': 'ModuleTempMin', 'value': {'stringValue': 
'20'}}, {'key': 'DCChargingEnergyIn', 'value': {'stringValue': '17.279999613761902'}}, {'key': 'InsideTemp', 
'value': {'stringValue': '19.50000088661909'}}, {'key': 'CarType', 'value': {'stringValue': 'Model3'}}, {'key': 
'ChargeEnableRequest', 'value': {'stringValue': 'true'}}, {'key': 'HvacPower', 'value': {'hvacPowerValue': 
'HvacPowerStateOff'}}, {'key': 'VehicleSpeed', 'value': {'invalid': True}}, {'key': 'IsolationResistance', 'value':
{'stringValue': '4290'}}, {'key': 'PackCurrent', 'value': {'stringValue': '0'}}, {'key': 'SentryMode', 'value': 
{'stringValue': 'Off'}}, {'key': 'ChargeCurrentRequest', 'value': {'stringValue': '16'}}, {'key': 
'ACChargingEnergyIn', 'value': {'stringValue': '19.64266695936988'}}, {'key': 'BatteryHeaterOn', 'value': 
{'stringValue': 'false'}}, {'key': 'DefrostForPreconditioning', 'value': {'booleanValue': False}}, {'key': 
'BrickVoltageMax', 'value': {'stringValue': '3.8940001849550754'}}, {'key': 'RearDefrostEnabled', 'value': 
{'booleanValue': False}}, {'key': 'ChargeLimitSoc', 'value': {'stringValue': '60'}}, {'key': 'DCDCEnable', 'value':
{'stringValue': 'false'}}, {'key': 'RatedRange', 'value': {'stringValue': '154.77550674457942'}}], 'meta': {}, 
'createdAt': '2025-05-21T17:12:05.612696221Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747847527112, 
'readable_date': '2025-05-21 17:12:07', 'data': [{'key': 'ChargerVoltage', 'value': {'doubleValue': 1.683}}], 
'meta': {}, 'createdAt': '2025-05-21T17:12:07.112413744Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747847555611, 'readable_date': '2025-05-21 17:12:35', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.800000011920929'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.4099916536361'}}, {'key': 
'LifetimeEnergyUsed', 'value': {'stringValue': '26561.661261611152'}}], 'meta': {}, 'createdAt': 
'2025-05-21T17:12:35.611996880Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747847561240, 'readable_date': 
'2025-05-21 17:12:41', 'data': [{'key': 'EnergyRemaining', 'value': {'stringValue': '39.519999116659164'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:12:41.240759172Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747847586240, 'readable_date': '2025-05-21 17:13:06', 'data': [{'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.663261611247'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, 
{'key': 'PackVoltage', 'value': {'stringValue': '373.53999165073037'}}], 'meta': {}, 'createdAt': 
'2025-05-21T17:13:06.240367865Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747847593240, 'readable_date': 
'2025-05-21 17:13:13', 'data': [{'key': 'BatteryLevel', 'value': {'stringValue': '60.4416403785489'}}], 'meta': {},
'createdAt': '2025-05-21T17:13:13.240721078Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747847616241, 
'readable_date': '2025-05-21 17:13:36', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.30000000447034836'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.449991652742'}}, {'key': 
'LifetimeEnergyUsed', 'value': {'stringValue': '26561.665261611342'}}], 'meta': {}, 'createdAt': 
'2025-05-21T17:13:36.241143775Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747847646240, 'readable_date': 
'2025-05-21 17:14:06', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 
'PackVoltage', 'value': {'stringValue': '373.2499916572124'}}], 'meta': {}, 'createdAt': 
'2025-05-21T17:14:06.240680699Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747847664240, 'readable_date': 
'2025-05-21 17:14:24', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': '26561.667261611437'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:14:24.240977588Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747847676243, 'readable_date': '2025-05-21 17:14:36', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.31999165564775'}}], 'meta': {}, 
'createdAt': '2025-05-21T17:14:36.243536537Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747847702740, 
'readable_date': '2025-05-21 17:15:02', 'data': [{'key': 'DCChargingEnergyIn', 'value': {'stringValue': 
'17.259999614208937'}}], 'meta': {}, 'createdAt': '2025-05-21T17:15:02.740617008Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747847703240, 'readable_date': '2025-05-21 17:15:03', 'data': [{'key': 'IdealBatteryRange', 'value': 
{'stringValue': '154.6938740933428'}}, {'key': 'Soc', 'value': {'stringValue': '59.779179810725545'}}], 'meta': {},
'createdAt': '2025-05-21T17:15:03.240301069Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747847706241, 
'readable_date': '2025-05-21 17:15:06', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.32999165542424'}}], 'meta': {}, 
'createdAt': '2025-05-21T17:15:06.241177399Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747847720241, 
'readable_date': '2025-05-21 17:15:20', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.669261611532'}}], 'meta': {}, 'createdAt': '2025-05-21T17:15:20.241130939Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747847736240, 'readable_date': '2025-05-21 17:15:36', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.53999165073037'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:15:36.240438266Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747847766240, 'readable_date': '2025-05-21 17:16:06', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.3999916538596'}}], 'meta': {}, 
'createdAt': '2025-05-21T17:16:06.240870762Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747847778240, 
'readable_date': '2025-05-21 17:16:18', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.671261611627'}}], 'meta': {}, 'createdAt': '2025-05-21T17:16:18.240961867Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747847796240, 'readable_date': '2025-05-21 17:16:36', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.31999165564775'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:16:36.240652686Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747847826240, 'readable_date': '2025-05-21 17:17:06', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.3399916552007'}}], 'meta': {}, 
'createdAt': '2025-05-21T17:17:06.240962821Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747847837241, 
'readable_date': '2025-05-21 17:17:17', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.675261611817'}}], 'meta': {}, 'createdAt': '2025-05-21T17:17:17.241022377Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747847856240, 'readable_date': '2025-05-21 17:17:36', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.5199916511774'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:17:36.240433374Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747847886241, 'readable_date': '2025-05-21 17:18:06', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.21999165788293'}}], 'meta': {}, 
'createdAt': '2025-05-21T17:18:06.241448953Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747847896240, 
'readable_date': '2025-05-21 17:18:16', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.677261611912'}}], 'meta': {}, 'createdAt': '2025-05-21T17:18:16.240846438Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747847916240, 'readable_date': '2025-05-21 17:18:36', 'data': [{'key': 'PackVoltage', 'value': 
{'stringValue': '373.42999165318906'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:18:36.240316961Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747847946240, 'readable_date': '2025-05-21 17:19:06', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': 
'373.36999165453017'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.30000000447034836'}}], 'meta': {}, 
'createdAt': '2025-05-21T17:19:06.240706618Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747847955241, 
'readable_date': '2025-05-21 17:19:15', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.679261612007'}}], 'meta': {}, 'createdAt': '2025-05-21T17:19:15.241017969Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747847976241, 'readable_date': '2025-05-21 17:19:36', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.30000000447034836'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.32999165542424'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:19:36.241221848Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747848003240, 'readable_date': '2025-05-21 17:20:03', 'data': [{'key': 'EnergyRemaining', 'value': {'stringValue':
'39.4999991171062'}}], 'meta': {}, 'createdAt': '2025-05-21T17:20:03.240654687Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747848006240, 'readable_date': '2025-05-21 17:20:06', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.30000000447034836'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.43999165296555'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:20:06.240599166Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747848015240, 'readable_date': '2025-05-21 17:20:15', 'data': [{'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.681261612102'}}], 'meta': {}, 'createdAt': '2025-05-21T17:20:15.240867838Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747848036240, 'readable_date': '2025-05-21 17:20:36', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.38999165408313'}}], 'meta': {}, 'createdAt': '2025-05-21T17:20:36.240336240Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747848051240, 'readable_date': '2025-05-21 17:20:51', 'data': [{'key': 'BatteryLevel', 'value': 
{'stringValue': '60.41009463722398'}}], 'meta': {}, 'createdAt': '2025-05-21T17:20:51.240429174Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747848066241, 'readable_date': '2025-05-21 17:21:06', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.30000000447034836'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.3499916549772'}}], 'meta': {}, 'createdAt': '2025-05-21T17:21:06.241363685Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747848075240, 'readable_date': '2025-05-21 17:21:15', 'data': [{'key': 'LifetimeEnergyUsed', 'value':
{'stringValue': '26561.683261612197'}}], 'meta': {}, 'createdAt': '2025-05-21T17:21:15.240897714Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747848096241, 'readable_date': '2025-05-21 17:21:36', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.5299916509539'}}], 'meta': {}, 'createdAt': '2025-05-21T17:21:36.241010025Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747848126240, 'readable_date': '2025-05-21 17:22:06', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.0499916616827'}}, 
{'key': 'BMSState', 'value': {'stringValue': 'Support'}}, {'key': 'ModuleTempMax', 'value': {'stringValue': 
'21'}}], 'meta': {}, 'createdAt': '2025-05-21T17:22:06.240992610Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747848135240, 'readable_date': '2025-05-21 17:22:15', 'data': [{'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.685261612292'}}], 'meta': {}, 'createdAt': '2025-05-21T17:22:15.240463006Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747848156240, 'readable_date': '2025-05-21 17:22:36', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.4599916525185'}}], 'meta': {}, 'createdAt': '2025-05-21T17:22:36.240991477Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747848160740, 'readable_date': '2025-05-21 17:22:40', 'data': [{'key': 'DCChargingEnergyIn', 'value':
{'stringValue': '17.23999961465597'}}], 'meta': {}, 'createdAt': '2025-05-21T17:22:40.740990350Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747848161240, 'readable_date': '2025-05-21 17:22:41', 'data': [{'key': 
'IdealBatteryRange', 'value': {'stringValue': '154.6122414421062'}}, {'key': 'Soc', 'value': {'stringValue': 
'59.74763406940063'}}], 'meta': {}, 'createdAt': '2025-05-21T17:22:41.240706992Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747848571669, 'readable_date': '2025-05-21 17:29:31', 'data': [{'key': 'HvacAutoMode', 'value': 
{'hvacAutoModeValue': 'HvacAutoModeStateOn'}}, {'key': 'InsideTemp', 'value': {'stringValue': 
'19.400000885128975'}}, {'key': 'FastChargerPresent', 'value': {'stringValue': 'false'}}, {'key': 'PackCurrent', 
'value': {'stringValue': '0'}}, {'key': 'ChargeAmps', 'value': {'stringValue': '0'}}, {'key': 
'IsolationResistance', 'value': {'stringValue': '4290'}}, {'key': 'EstBatteryRange', 'value': {'stringValue': 
'138.09868713826083'}}, {'key': 'EnergyRemaining', 'value': {'stringValue': '39.4999991171062'}}, {'key': 
'ChargePortColdWeatherMode', 'value': {'stringValue': 'false'}}, {'key': 'ACChargingPower', 'value': 
{'stringValue': '0'}}, {'key': 'IdealBatteryRange', 'value': {'stringValue': '154.6122414421062'}}, {'key': 
'ModuleTempMax', 'value': {'stringValue': '20.5'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.5299916509539'}}, {'key': 'LifetimeEnergyUsed', 'value': {'stringValue': '26561.685261612292'}}, {'key': 
'DCDCEnable', 'value': {'stringValue': 'false'}}, {'key': 'BatteryLevel', 'value': {'stringValue': 
'60.41009463722398'}}, {'key': 'Odometer', 'value': {'stringValue': '63928.977202334645'}}, {'key': 'SentryMode', 
'value': {'stringValue': 'Off'}}, {'key': 'OutsideTemp', 'value': {'stringValue': '19'}}, {'key': 
'DCChargingPower', 'value': {'stringValue': '0'}}, {'key': 'ChargerPhases', 'value': {'invalid': True}}, {'key': 
'DefrostForPreconditioning', 'value': {'booleanValue': False}}, {'key': 'BmsFullchargecomplete', 'value': 
{'stringValue': 'true'}}, {'key': 'DefrostMode', 'value': {'defrostModeValue': 'DefrostModeStateOff'}}, {'key': 
'ACChargingEnergyIn', 'value': {'stringValue': '19.64266695936988'}}, {'key': 'ChargingCableType', 'value': 
{'cableTypeValue': 'CableTypeIEC'}}, {'key': 'ChargePort', 'value': {'stringValue': 'CCS'}}, {'key': 
'ModuleTempMin', 'value': {'stringValue': '20'}}, {'key': 'ClimateKeeperMode', 'value': {'climateKeeperModeValue': 
'ClimateKeeperModeStateOff'}}, {'key': 'HvacPower', 'value': {'hvacPowerValue': 'HvacPowerStateOff'}}, {'key': 
'ChargeCurrentRequest', 'value': {'stringValue': '16'}}, {'key': 'ChargeLimitSoc', 'value': {'stringValue': '60'}},
{'key': 'Soc', 'value': {'stringValue': '59.74763406940063'}}, {'key': 'DetailedChargeState', 'value': 
{'detailedChargeStateValue': 'DetailedChargeStateComplete'}}, {'key': 'ChargeCurrentRequestMax', 'value': 
{'stringValue': '16'}}, {'key': 'RatedRange', 'value': {'stringValue': '154.6122414421062'}}, {'key': 
'BrickVoltageMax', 'value': {'stringValue': '3.8940001849550754'}}, {'key': 'ChargeState', 'value': {'stringValue':
'Idle'}}, {'key': 'RearDefrostEnabled', 'value': {'booleanValue': False}}, {'key': 'BMSState', 'value': 
{'stringValue': 'Standby'}}, {'key': 'BrickVoltageMin', 'value': {'stringValue': '3.8920001848600805'}}, {'key': 
'EstimatedHoursToChargeTermination', 'value': {'invalid': True}}, {'key': 'VehicleSpeed', 'value': {'invalid': 
True}}, {'key': 'ChargeRateMilePerHour', 'value': {'doubleValue': 0}}, {'key': 'FastChargerType', 'value': 
{'invalid': True}}, {'key': 'EuropeVehicle', 'value': {'booleanValue': True}}, {'key': 'EfficiencyPackage', 
'value': {'stringValue': 'Default'}}, {'key': 'ChargeEnableRequest', 'value': {'stringValue': 'true'}}, {'key': 
'HvacACEnabled', 'value': {'booleanValue': False}}, {'key': 'PreconditioningEnabled', 'value': {'stringValue': 
'false'}}, {'key': 'CarType', 'value': {'stringValue': 'Model3'}}, {'key': 'HvacFanSpeed', 'value': {'intValue': 
2}}, {'key': 'DCChargingEnergyIn', 'value': {'stringValue': '17.23999961465597'}}, {'key': 'BatteryHeaterOn', 
'value': {'stringValue': 'false'}}], 'meta': {}, 'createdAt': '2025-05-21T17:29:31.669610054Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747848573168, 'readable_date': '2025-05-21 17:29:33', 'data': [{'key': 
'ChargerVoltage', 'value': {'doubleValue': 1.716}}], 'meta': {}, 'createdAt': '2025-05-21T17:29:33.168815195Z'}, 
{'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747848601669, 'readable_date': '2025-05-21 17:30:01', 'data': [{'key': 
'LifetimeEnergyUsed', 'value': {'stringValue': '26561.687261612387'}}, {'key': 'PackCurrent', 'value': 
{'stringValue': '-0.800000011920929'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.2799916565418'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:30:01.669640161Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747848632086, 'readable_date': '2025-05-21 17:30:32', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': 
'373.5199916511774'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 
'LifetimeEnergyUsed', 'value': {'stringValue': '26561.691261612577'}}], 'meta': {}, 'createdAt': 
'2025-05-21T17:30:32.086564802Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747848662087, 'readable_date': 
'2025-05-21 17:31:02', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 
'PackVoltage', 'value': {'stringValue': '373.49999165162444'}}], 'meta': {}, 'createdAt': 
'2025-05-21T17:31:02.087102889Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747848679586, 'readable_date': 
'2025-05-21 17:31:19', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': '26561.693261612672'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:31:19.586959338Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747848692086, 'readable_date': '2025-05-21 17:31:32', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.30000000447034836'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.32999165542424'}}], 'meta': {}, 
'createdAt': '2025-05-21T17:31:32.086500761Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747848709586, 
'readable_date': '2025-05-21 17:31:49', 'data': [{'key': 'EnergyRemaining', 'value': {'stringValue': 
'39.479999117553234'}}], 'meta': {}, 'createdAt': '2025-05-21T17:31:49.586357888Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747848722086, 'readable_date': '2025-05-21 17:32:02', 'data': [{'key': 'PackVoltage', 'value': 
{'stringValue': '373.1899916585535'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:32:02.086337577Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747848738586, 'readable_date': '2025-05-21 17:32:18', 'data': [{'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.695261612767'}}], 'meta': {}, 'createdAt': '2025-05-21T17:32:18.586449413Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747848752086, 'readable_date': '2025-05-21 17:32:32', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.32999165542424'}}], 'meta': {}, 'createdAt': '2025-05-21T17:32:32.086558160Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747848782087, 'readable_date': '2025-05-21 17:33:02', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.3599916547537'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:33:02.087227383Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747848798586, 'readable_date': '2025-05-21 17:33:18', 'data': [{'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.697261612862'}}], 'meta': {}, 'createdAt': '2025-05-21T17:33:18.586822765Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747848812086, 'readable_date': '2025-05-21 17:33:32', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.2399916574359'}}], 'meta': {}, 'createdAt': '2025-05-21T17:33:32.086141867Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747848842086, 'readable_date': '2025-05-21 17:34:02', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.30000000447034836'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.38999165408313'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:34:02.086357775Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747848859586, 'readable_date': '2025-05-21 17:34:19', 'data': [{'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.699261612957'}}], 'meta': {}, 'createdAt': '2025-05-21T17:34:19.586445385Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747848872085, 'readable_date': '2025-05-21 17:34:32', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.2999916560948'}}], 'meta': {}, 'createdAt': '2025-05-21T17:34:32.085997365Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747848884586, 'readable_date': '2025-05-21 17:34:44', 'data': [{'key': 'BatteryLevel', 'value': 
{'stringValue': '60.37854889589905'}}], 'meta': {}, 'createdAt': '2025-05-21T17:34:44.586620647Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747848887587, 'readable_date': '2025-05-21 17:34:47', 'data': [{'key': 
'EnergyRemaining', 'value': {'stringValue': '39.45999911800027'}}], 'meta': {}, 'createdAt': 
'2025-05-21T17:34:47.587135016Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747848902086, 'readable_date': 
'2025-05-21 17:35:02', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': '373.5299916509539'}}, {'key': 
'PackCurrent', 'value': {'stringValue': '-0.30000000447034836'}}], 'meta': {}, 'createdAt': 
'2025-05-21T17:35:02.086393948Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747848919586, 'readable_date': 
'2025-05-21 17:35:19', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': '26561.701261613052'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:35:19.586460455Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747848932086, 'readable_date': '2025-05-21 17:35:32', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': 
'373.31999165564775'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.30000000447034836'}}], 'meta': {}, 
'createdAt': '2025-05-21T17:35:32.086953453Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747848962086, 
'readable_date': '2025-05-21 17:36:02', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.31999165564775'}}], 'meta': {}, 
'createdAt': '2025-05-21T17:36:02.086043493Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747848980586, 
'readable_date': '2025-05-21 17:36:20', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.705261613242'}}], 'meta': {}, 'createdAt': '2025-05-21T17:36:20.586911883Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747848992086, 'readable_date': '2025-05-21 17:36:32', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.2499916572124'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:36:32.086117971Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747849002586, 'readable_date': '2025-05-21 17:36:42', 'data': [{'key': 'IdealBatteryRange', 'value': 
{'stringValue': '154.5306087908696'}}, {'key': 'Soc', 'value': {'stringValue': '59.71608832807571'}}, {'key': 
'DCChargingEnergyIn', 'value': {'stringValue': '17.219999615103006'}}], 'meta': {}, 'createdAt': 
'2025-05-21T17:36:42.586557823Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747849022086, 'readable_date': 
'2025-05-21 17:37:02', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': '373.2399916574359'}}, {'key': 
'PackCurrent', 'value': {'stringValue': '-0.30000000447034836'}}], 'meta': {}, 'createdAt': 
'2025-05-21T17:37:02.086604126Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747849040586, 'readable_date': 
'2025-05-21 17:37:20', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': '26561.707261613337'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:37:20.586587788Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747849052086, 'readable_date': '2025-05-21 17:37:32', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': 
'373.0199916623533'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.30000000447034836'}}], 'meta': {}, 
'createdAt': '2025-05-21T17:37:32.086164226Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747849082086, 
'readable_date': '2025-05-21 17:38:02', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.30000000447034836'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.6499916482717'}}], 'meta': {}, 
'createdAt': '2025-05-21T17:38:02.086027874Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747849101586, 
'readable_date': '2025-05-21 17:38:21', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.709261613432'}}], 'meta': {}, 'createdAt': '2025-05-21T17:38:21.586858242Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747849112086, 'readable_date': '2025-05-21 17:38:32', 'data': [{'key': 'PackVoltage', 'value': 
{'stringValue': '373.32999165542424'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:38:32.086162928Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747849142086, 'readable_date': '2025-05-21 17:39:02', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.30000000447034836'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.25999165698886'}}], 'meta': {}, 
'createdAt': '2025-05-21T17:39:02.086084787Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747849162586, 
'readable_date': '2025-05-21 17:39:22', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.711261613527'}}], 'meta': {}, 'createdAt': '2025-05-21T17:39:22.586745470Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747849172086, 'readable_date': '2025-05-21 17:39:32', 'data': [{'key': 'PackVoltage', 'value': 
{'stringValue': '372.9999916628003'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, 
{'key': 'BMSState', 'value': {'stringValue': 'Support'}}, {'key': 'ModuleTempMax', 'value': {'stringValue': 
'21'}}], 'meta': {}, 'createdAt': '2025-05-21T17:39:32.086264496Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747849185587, 'readable_date': '2025-05-21 17:39:45', 'data': [{'key': 'EnergyRemaining', 'value': {'stringValue':
'39.439999118447304'}}], 'meta': {}, 'createdAt': '2025-05-21T17:39:45.587237971Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747849202086, 'readable_date': '2025-05-21 17:40:02', 'data': [{'key': 'PackVoltage', 'value': 
{'stringValue': '373.54999165050685'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}], 
'meta': {}, 'createdAt': '2025-05-21T17:40:02.086279470Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747861694408, 'readable_date': '2025-05-21 21:08:14', 'data': [{'key': 'BmsFullchargecomplete', 'value': 
{'stringValue': 'true'}}, {'key': 'ModuleTempMax', 'value': {'stringValue': '21'}}, {'key': 'ModuleTempMin', 
'value': {'stringValue': '20'}}, {'key': 'HvacAutoMode', 'value': {'hvacAutoModeValue': 'HvacAutoModeStateOn'}}, 
{'key': 'OutsideTemp', 'value': {'stringValue': '19'}}, {'key': 'ChargePortColdWeatherMode', 'value': 
{'stringValue': 'false'}}, {'key': 'FastChargerPresent', 'value': {'stringValue': 'false'}}, {'key': 'PackCurrent',
'value': {'stringValue': '0'}}, {'key': 'ChargeEnableRequest', 'value': {'stringValue': 'true'}}, {'key': 
'ChargeCurrentRequest', 'value': {'stringValue': '16'}}, {'key': 'InsideTemp', 'value': {'stringValue': 
'19.50000088661909'}}, {'key': 'BrickVoltageMax', 'value': {'stringValue': '3.8940001849550754'}}, {'key': 
'LifetimeEnergyUsed', 'value': {'stringValue': '26561.711261613527'}}, {'key': 'ACChargingEnergyIn', 'value': 
{'stringValue': '19.64266695936988'}}, {'key': 'ChargingCableType', 'value': {'cableTypeValue': 'CableTypeIEC'}}, 
{'key': 'CarType', 'value': {'stringValue': 'Model3'}}, {'key': 'DCDCEnable', 'value': {'stringValue': 'false'}}, 
{'key': 'PreconditioningEnabled', 'value': {'stringValue': 'false'}}, {'key': 'ChargeAmps', 'value': 
{'stringValue': '0'}}, {'key': 'ChargePort', 'value': {'stringValue': 'CCS'}}, {'key': 'ChargeLimitSoc', 'value': 
{'stringValue': '60'}}, {'key': 'DCChargingEnergyIn', 'value': {'stringValue': '17.219999615103006'}}, {'key': 
'RatedRange', 'value': {'stringValue': '154.5306087908696'}}, {'key': 'EstBatteryRange', 'value': {'stringValue': 
'138.02577336469258'}}, {'key': 'EfficiencyPackage', 'value': {'stringValue': 'Default'}}, {'key': 'HvacFanSpeed', 
'value': {'intValue': 2}}, {'key': 'DefrostForPreconditioning', 'value': {'booleanValue': False}}, {'key': 
'PackVoltage', 'value': {'stringValue': '373.3099916558713'}}, {'key': 'IsolationResistance', 'value': 
{'stringValue': '4290'}}, {'key': 'ACChargingPower', 'value': {'stringValue': '0'}}, {'key': 'Soc', 'value': 
{'stringValue': '59.71608832807571'}}, {'key': 'DefrostMode', 'value': {'defrostModeValue': 
'DefrostModeStateOff'}}, {'key': 'RearDefrostEnabled', 'value': {'booleanValue': False}}, {'key': 
'FastChargerType', 'value': {'invalid': True}}, {'key': 'VehicleSpeed', 'value': {'invalid': True}}, {'key': 
'EnergyRemaining', 'value': {'stringValue': '39.439999118447304'}}, {'key': 'ChargeCurrentRequestMax', 'value': 
{'stringValue': '16'}}, {'key': 'BatteryHeaterOn', 'value': {'stringValue': 'false'}}, {'key': 'SentryMode', 
'value': {'stringValue': 'Off'}}, {'key': 'Odometer', 'value': {'stringValue': '63928.977202334645'}}, {'key': 
'HvacACEnabled', 'value': {'booleanValue': False}}, {'key': 'DCChargingPower', 'value': {'stringValue': '0'}}, 
{'key': 'IdealBatteryRange', 'value': {'stringValue': '154.5306087908696'}}, {'key': 'DetailedChargeState', 
'value': {'detailedChargeStateValue': 'DetailedChargeStateComplete'}}, {'key': 'ChargeRateMilePerHour', 'value': 
{'doubleValue': 0}}, {'key': 'BMSState', 'value': {'stringValue': 'Standby'}}, {'key': 'EuropeVehicle', 'value': 
{'booleanValue': True}}, {'key': 'BrickVoltageMin', 'value': {'stringValue': '3.8900001847650856'}}, {'key': 
'ChargeState', 'value': {'stringValue': 'Idle'}}, {'key': 'ChargerPhases', 'value': {'invalid': True}}, {'key': 
'BatteryLevel', 'value': {'stringValue': '60.37854889589905'}}, {'key': 'ClimateKeeperMode', 'value': 
{'climateKeeperModeValue': 'ClimateKeeperModeStateOff'}}, {'key': 'HvacPower', 'value': {'hvacPowerValue': 
'HvacPowerStateOff'}}, {'key': 'EstimatedHoursToChargeTermination', 'value': {'invalid': True}}], 'meta': {}, 
'createdAt': '2025-05-21T21:08:14.408022251Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747861695903, 
'readable_date': '2025-05-21 21:08:15', 'data': [{'key': 'ChargerVoltage', 'value': {'doubleValue': 1.716}}], 
'meta': {}, 'createdAt': '2025-05-21T21:08:15.903014372Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747861724402, 'readable_date': '2025-05-21 21:08:44', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': 
'373.2299916576594'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.9000000134110451'}}, {'key': 
'LifetimeEnergyUsed', 'value': {'stringValue': '26561.719261613907'}}], 'meta': {}, 'createdAt': 
'2025-05-21T21:08:44.402743990Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747861727403, 'readable_date': 
'2025-05-21 21:08:47', 'data': [{'key': 'BatteryLevel', 'value': {'stringValue': '60.34700315457413'}}], 'meta': 
{}, 'createdAt': '2025-05-21T21:08:47.403371392Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747861754693, 
'readable_date': '2025-05-21 21:09:14', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.26999165676534'}}, {'key': 
'LifetimeEnergyUsed', 'value': {'stringValue': '26561.721261614002'}}], 'meta': {}, 'createdAt': 
'2025-05-21T21:09:14.693213051Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747861784693, 'readable_date': 
'2025-05-21 21:09:44', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': '373.2899916563183'}}, {'key': 
'PackCurrent', 'value': {'stringValue': '-0.30000000447034836'}}], 'meta': {}, 'createdAt': 
'2025-05-21T21:09:44.693776150Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747861787692, 'readable_date': 
'2025-05-21 21:09:47', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': '26561.723261614097'}}], 
'meta': {}, 'createdAt': '2025-05-21T21:09:47.692889726Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747861795193, 'readable_date': '2025-05-21 21:09:55', 'data': [{'key': 'DCChargingEnergyIn', 'value': 
{'stringValue': '17.19999961555004'}}], 'meta': {}, 'createdAt': '2025-05-21T21:09:55.193811975Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747861795693, 'readable_date': '2025-05-21 21:09:55', 'data': [{'key': 'Soc', 
'value': {'stringValue': '59.68454258675079'}}, {'key': 'IdealBatteryRange', 'value': {'stringValue': 
'154.448976139633'}}], 'meta': {}, 'createdAt': '2025-05-21T21:09:55.693645183Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747861814693, 'readable_date': '2025-05-21 21:10:14', 'data': [{'key': 'PackVoltage', 'value': 
{'stringValue': '373.15999165922403'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.5000000074505806'}}], 
'meta': {}, 'createdAt': '2025-05-21T21:10:14.693810102Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747861841693, 'readable_date': '2025-05-21 21:10:41', 'data': [{'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.725261614192'}}], 'meta': {}, 'createdAt': '2025-05-21T21:10:41.693607412Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747861844693, 'readable_date': '2025-05-21 21:10:44', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.179991658777'}}], 'meta': {}, 'createdAt': '2025-05-21T21:10:44.693589249Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747861874693, 'readable_date': '2025-05-21 21:11:14', 'data': [{'key': 'PackVoltage', 'value': 
{'stringValue': '373.13999165967107'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}], 
'meta': {}, 'createdAt': '2025-05-21T21:11:14.693594343Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747861896693, 'readable_date': '2025-05-21 21:11:36', 'data': [{'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.727261614287'}}], 'meta': {}, 'createdAt': '2025-05-21T21:11:36.693174316Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747861904693, 'readable_date': '2025-05-21 21:11:44', 'data': [{'key': 
'PackVoltage', 'value': {'stringValue': '373.25999165698886'}}, {'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}], 'meta': {}, 'createdAt': '2025-05-21T21:11:44.693493670Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747861934692, 'readable_date': '2025-05-21 21:12:14', 'data': [{'key': 'PackVoltage', 'value': 
{'stringValue': '373.1899916585535'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}], 
'meta': {}, 'createdAt': '2025-05-21T21:12:14.692905325Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747861953693, 'readable_date': '2025-05-21 21:12:33', 'data': [{'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.729261614382'}}], 'meta': {}, 'createdAt': '2025-05-21T21:12:33.693783372Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747861964693, 'readable_date': '2025-05-21 21:12:44', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.1099916603416'}}], 'meta': {}, 'createdAt': '2025-05-21T21:12:44.693157580Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747861994693, 'readable_date': '2025-05-21 21:13:14', 'data': [{'key': 'PackVoltage', 'value': 
{'stringValue': '373.19999165832996'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}], 
'meta': {}, 'createdAt': '2025-05-21T21:13:14.693324879Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747862011692, 'readable_date': '2025-05-21 21:13:31', 'data': [{'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.733261614572'}}], 'meta': {}, 'createdAt': '2025-05-21T21:13:31.692875960Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747862024692, 'readable_date': '2025-05-21 21:13:44', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.38999165408313'}}], 'meta': {}, 'createdAt': '2025-05-21T21:13:44.692871396Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747862028693, 'readable_date': '2025-05-21 21:13:48', 'data': [{'key': 'EnergyRemaining', 'value': 
{'stringValue': '39.41999911889434'}}], 'meta': {}, 'createdAt': '2025-05-21T21:13:48.693216332Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747862054693, 'readable_date': '2025-05-21 21:14:14', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.1699916590005'}}], 'meta': {}, 'createdAt': '2025-05-21T21:14:14.693809618Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747862070693, 'readable_date': '2025-05-21 21:14:30', 'data': [{'key': 'LifetimeEnergyUsed', 'value':
{'stringValue': '26561.735261614667'}}], 'meta': {}, 'createdAt': '2025-05-21T21:14:30.693804259Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747862084693, 'readable_date': '2025-05-21 21:14:44', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'373.2999916560948'}}], 'meta': {}, 'createdAt': '2025-05-21T21:14:44.693815115Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747862114693, 'readable_date': '2025-05-21 21:15:14', 'data': [{'key': 'PackVoltage', 'value': 
{'stringValue': '373.1699916590005'}}, {'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}], 
'meta': {}, 'createdAt': '2025-05-21T21:15:14.693505250Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747862129693, 'readable_date': '2025-05-21 21:15:29', 'data': [{'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.737261614762'}}], 'meta': {}, 'createdAt': '2025-05-21T21:15:29.693116134Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747862135693, 'readable_date': '2025-05-21 21:15:35', 'data': [{'key': 
'BatteryLevel', 'value': {'stringValue': '60.315457413249206'}}], 'meta': {}, 'createdAt': 
'2025-05-21T21:15:35.693824023Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747862144693, 'readable_date': 
'2025-05-21 21:15:44', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}, {'key': 
'PackVoltage', 'value': {'stringValue': '373.03999166190624'}}], 'meta': {}, 'createdAt': 
'2025-05-21T21:15:44.693655948Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747862174693, 'readable_date': 
'2025-05-21 21:16:14', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': '373.03999166190624'}}, {'key': 
'PackCurrent', 'value': {'stringValue': '-0.4000000059604645'}}], 'meta': {}, 'createdAt': 
'2025-05-21T21:16:14.693831048Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747862189693, 'readable_date': 
'2025-05-21 21:16:29', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': '26561.739261614857'}}], 
'meta': {}, 'createdAt': '2025-05-21T21:16:29.693602193Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747862204693, 'readable_date': '2025-05-21 21:16:44', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.30000000447034836'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.37999165430665'}}], 'meta': {}, 
'createdAt': '2025-05-21T21:16:44.693111281Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747862234692, 
'readable_date': '2025-05-21 21:17:14', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.179991658777'}}], 'meta': {}, 
'createdAt': '2025-05-21T21:17:14.692908856Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747862245193, 
'readable_date': '2025-05-21 21:17:25', 'data': [{'key': 'DCChargingEnergyIn', 'value': {'stringValue': 
'17.179999615997076'}}], 'meta': {}, 'createdAt': '2025-05-21T21:17:25.193771289Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747862245693, 'readable_date': '2025-05-21 21:17:25', 'data': [{'key': 'IdealBatteryRange', 'value': 
{'stringValue': '154.36734348839641'}}, {'key': 'Soc', 'value': {'stringValue': '59.65299684542587'}}], 'meta': {},
'createdAt': '2025-05-21T21:17:25.693794965Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747862264693, 
'readable_date': '2025-05-21 21:17:44', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.32999165542424'}}], 'meta': {}, 
'createdAt': '2025-05-21T21:17:44.693745892Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747862294693, 
'readable_date': '2025-05-21 21:18:14', 'data': [{'key': 'BMSState', 'value': {'stringValue': 'Support'}}, {'key': 
'ModuleTempMax', 'value': {'stringValue': '21'}}, {'key': 'PackCurrent', 'value': {'stringValue': 
'-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.02999166212976'}}], 'meta': {}, 
'createdAt': '2025-05-21T21:18:14.693532451Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747862309693, 
'readable_date': '2025-05-21 21:18:29', 'data': [{'key': 'LifetimeEnergyUsed', 'value': {'stringValue': 
'26561.741261614952'}}], 'meta': {}, 'createdAt': '2025-05-21T21:18:29.693718630Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747862324692, 'readable_date': '2025-05-21 21:18:44', 'data': [{'key': 'PackCurrent', 'value': 
{'stringValue': '-0.4000000059604645'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.2799916565418'}}], 
'meta': {}, 'createdAt': '2025-05-21T21:18:44.692903834Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747871685565, 'readable_date': '2025-05-21 23:54:45', 'data': [{'key': 'FastChargerPresent', 'value': 
{'stringValue': 'false'}}, {'key': 'Odometer', 'value': {'stringValue': '63928.977202334645'}}, {'key': 
'HvacACEnabled', 'value': {'booleanValue': False}}, {'key': 'ChargingCableType', 'value': {'cableTypeValue': 
'CableTypeIEC'}}, {'key': 'EstBatteryRange', 'value': {'stringValue': '137.8799458175561'}}, {'key': 
'ChargeEnableRequest', 'value': {'stringValue': 'true'}}, {'key': 'PreconditioningEnabled', 'value': 
{'stringValue': 'false'}}, {'key': 'SentryMode', 'value': {'stringValue': 'Off'}}, {'key': 
'EstimatedHoursToChargeTermination', 'value': {'invalid': True}}, {'key': 'ChargePort', 'value': {'stringValue': 
'CCS'}}, {'key': 'EuropeVehicle', 'value': {'booleanValue': True}}, {'key': 'ChargeLimitSoc', 'value': 
{'stringValue': '60'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.449991652742'}}, {'key': 
'ChargeCurrentRequestMax', 'value': {'stringValue': '16'}}, {'key': 'ChargePortColdWeatherMode', 'value': 
{'stringValue': 'false'}}, {'key': 'ChargerPhases', 'value': {'invalid': True}}, {'key': 'ChargeCurrentRequest', 
'value': {'stringValue': '16'}}, {'key': 'HvacPower', 'value': {'hvacPowerValue': 'HvacPowerStateOff'}}, {'key': 
'DCDCEnable', 'value': {'stringValue': 'false'}}, {'key': 'ChargeState', 'value': {'stringValue': 'Idle'}}, {'key':
'BrickVoltageMin', 'value': {'stringValue': '3.8900001847650856'}}, {'key': 'ACChargingPower', 'value': 
{'stringValue': '0'}}, {'key': 'Soc', 'value': {'stringValue': '59.65299684542587'}}, {'key': 'VehicleSpeed', 
'value': {'invalid': True}}, {'key': 'EfficiencyPackage', 'value': {'stringValue': 'Default'}}, {'key': 
'ModuleTempMax', 'value': {'stringValue': '21'}}, {'key': 'DCChargingEnergyIn', 'value': {'stringValue': 
'17.179999615997076'}}, {'key': 'BMSState', 'value': {'stringValue': 'Standby'}}, {'key': 
'DefrostForPreconditioning', 'value': {'booleanValue': False}}, {'key': 'ChargeRateMilePerHour', 'value': 
{'doubleValue': 0}}, {'key': 'LifetimeEnergyUsed', 'value': {'stringValue': '26561.745261615142'}}, {'key': 
'ACChargingEnergyIn', 'value': {'stringValue': '19.64266695936988'}}, {'key': 'InsideTemp', 'value': 
{'stringValue': '19.400000885128975'}}, {'key': 'BatteryLevel', 'value': {'stringValue': '60.315457413249206'}}, 
{'key': 'BatteryHeaterOn', 'value': {'stringValue': 'false'}}, {'key': 'IdealBatteryRange', 'value': 
{'stringValue': '154.36734348839641'}}, {'key': 'BmsFullchargecomplete', 'value': {'stringValue': 'true'}}, {'key':
'DefrostMode', 'value': {'defrostModeValue': 'DefrostModeStateOff'}}, {'key': 'RearDefrostEnabled', 'value': 
{'booleanValue': False}}, {'key': 'HvacFanSpeed', 'value': {'intValue': 2}}, {'key': 'OutsideTemp', 'value': 
{'stringValue': '19'}}, {'key': 'BrickVoltageMax', 'value': {'stringValue': '3.8920001848600805'}}, {'key': 
'RatedRange', 'value': {'stringValue': '154.36734348839641'}}, {'key': 'PackCurrent', 'value': {'stringValue': 
'0'}}, {'key': 'ClimateKeeperMode', 'value': {'climateKeeperModeValue': 'ClimateKeeperModeStateOff'}}, {'key': 
'CarType', 'value': {'stringValue': 'Model3'}}, {'key': 'IsolationResistance', 'value': {'stringValue': '4290'}}, 
{'key': 'EnergyRemaining', 'value': {'stringValue': '39.41999911889434'}}, {'key': 'HvacAutoMode', 'value': 
{'hvacAutoModeValue': 'HvacAutoModeStateOn'}}, {'key': 'DCChargingPower', 'value': {'stringValue': '0'}}, {'key': 
'ModuleTempMin', 'value': {'stringValue': '20'}}, {'key': 'ChargeAmps', 'value': {'stringValue': '0'}}, {'key': 
'DetailedChargeState', 'value': {'detailedChargeStateValue': 'DetailedChargeStateComplete'}}, {'key': 
'FastChargerType', 'value': {'invalid': True}}], 'meta': {}, 'createdAt': '2025-05-21T23:54:45.565054692Z'}, 
{'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871686564, 'readable_date': '2025-05-21 23:54:46', 'data': [{'key': 
'ChargerVoltage', 'value': {'doubleValue': 1.683}}], 'meta': {}, 'createdAt': '2025-05-21T23:54:46.564293655Z'}, 
{'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871715563, 'readable_date': '2025-05-21 23:55:15', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '-0.5000000074505806'}}, {'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.747261615237'}}, {'key': 'PackVoltage', 'value': {'stringValue': '373.03999166190624'}}], 
'meta': {}, 'createdAt': '2025-05-21T23:55:15.563981396Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747871736064, 'readable_date': '2025-05-21 23:55:36', 'data': [{'key': 'ACChargingPower', 'value': {'stringValue':
'0.30000000447034836'}}], 'meta': {}, 'createdAt': '2025-05-21T23:55:36.064384504Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747871737564, 'readable_date': '2025-05-21 23:55:37', 'data': [{'key': 'ACChargingEnergyIn', 'value':
{'stringValue': '19.642722514926263'}}], 'meta': {}, 'createdAt': '2025-05-21T23:55:37.564275984Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747871740563, 'readable_date': '2025-05-21 23:55:40', 'data': [{'key': 
'ChargeRateMilePerHour', 'value': {'doubleValue': 0.07632653061224456}}], 'meta': {}, 'createdAt': 
'2025-05-21T23:55:40.563656556Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871745967, 'readable_date': 
'2025-05-21 23:55:45', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': '373.9199916422367'}}, {'key': 
'PackCurrent', 'value': {'stringValue': '8.700000129640102'}}, {'key': 'LifetimeEnergyUsed', 'value': 
{'stringValue': '26561.749261615332'}}], 'meta': {}, 'createdAt': '2025-05-21T23:55:45.967468963Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747871766467, 'readable_date': '2025-05-21 23:56:06', 'data': [{'key': 
'ACChargingPower', 'value': {'stringValue': '3.6000000536441803'}}], 'meta': {}, 'createdAt': 
'2025-05-21T23:56:06.467226509Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871767967, 'readable_date': 
'2025-05-21 23:56:07', 'data': [{'key': 'ACChargingEnergyIn', 'value': {'stringValue': '19.668833626426462'}}], 
'meta': {}, 'createdAt': '2025-05-21T23:56:07.967061015Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747871770967, 'readable_date': '2025-05-21 23:56:10', 'data': [{'key': 'ChargeRateMilePerHour', 'value': 
{'doubleValue': 13.35714285714316}}], 'meta': {}, 'createdAt': '2025-05-21T23:56:10.967555103Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747871772467, 'readable_date': '2025-05-21 23:56:12', 'data': [{'key': 
'EnergyRemaining', 'value': {'stringValue': '39.439999118447304'}}], 'meta': {}, 'createdAt': 
'2025-05-21T23:56:12.467184735Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871775966, 'readable_date': 
'2025-05-21 23:56:15', 'data': [{'key': 'PackCurrent', 'value': {'stringValue': '8.600000128149986'}}, {'key': 
'PackVoltage', 'value': {'stringValue': '374.1699916366488'}}], 'meta': {}, 'createdAt': 
'2025-05-21T23:56:15.966465889Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871776967, 'readable_date': 
'2025-05-21 23:56:16', 'data': [{'key': 'DCChargingEnergyIn', 'value': {'stringValue': '17.19999961555004'}}], 
'meta': {}, 'createdAt': '2025-05-21T23:56:16.967037832Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747871777467, 'readable_date': '2025-05-21 23:56:17', 'data': [{'key': 'IdealBatteryRange', 'value': 
{'stringValue': '154.448976139633'}}, {'key': 'Soc', 'value': {'stringValue': '59.68454258675079'}}], 'meta': {}, 
'createdAt': '2025-05-21T23:56:17.467301844Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871796467, 
'readable_date': '2025-05-21 23:56:36', 'data': [{'key': 'ACChargingPower', 'value': {'stringValue': 
'3.500000052154064'}}], 'meta': {}, 'createdAt': '2025-05-21T23:56:36.467341514Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747871797967, 'readable_date': '2025-05-21 23:56:37', 'data': [{'key': 'ACChargingEnergyIn', 'value':
{'stringValue': '19.69822251575329'}}], 'meta': {}, 'createdAt': '2025-05-21T23:56:37.967183050Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747871800967, 'readable_date': '2025-05-21 23:56:40', 'data': [{'key': 
'ChargeRateMilePerHour', 'value': {'doubleValue': 13.285102040816534}}], 'meta': {}, 'createdAt': 
'2025-05-21T23:56:40.967487541Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871802466, 'readable_date': 
'2025-05-21 23:56:42', 'data': [{'key': 'EnergyRemaining', 'value': {'stringValue': '39.45999911800027'}}], 'meta':
{}, 'createdAt': '2025-05-21T23:56:42.466698548Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871803467, 
'readable_date': '2025-05-21 23:56:43', 'data': [{'key': 'BatteryLevel', 'value': {'stringValue': 
'60.37854889589905'}}], 'meta': {}, 'createdAt': '2025-05-21T23:56:43.467241827Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747871805967, 'readable_date': '2025-05-21 23:56:45', 'data': [{'key': 'PackVoltage', 'value': 
{'stringValue': '374.45999163016677'}}, {'key': 'PackCurrent', 'value': {'stringValue': '8.600000128149986'}}], 
'meta': {}, 'createdAt': '2025-05-21T23:56:45.967278279Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747871826467, 'readable_date': '2025-05-21 23:57:06', 'data': [{'key': 'ACChargingPower', 'value': {'stringValue':
'3.6000000536441803'}}], 'meta': {}, 'createdAt': '2025-05-21T23:57:06.467431887Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747871826967, 'readable_date': '2025-05-21 23:57:06', 'data': [{'key': 'DCChargingEnergyIn', 'value':
{'stringValue': '17.219999615103006'}}], 'meta': {}, 'createdAt': '2025-05-21T23:57:06.967246092Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747871827466, 'readable_date': '2025-05-21 23:57:07', 'data': [{'key': 'Soc', 
'value': {'stringValue': '59.71608832807571'}}], 'meta': {}, 'createdAt': '2025-05-21T23:57:07.466735844Z'}, 
{'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871827967, 'readable_date': '2025-05-21 23:57:07', 'data': [{'key': 
'ACChargingEnergyIn', 'value': {'stringValue': '19.72761140508012'}}], 'meta': {}, 'createdAt': 
'2025-05-21T23:57:07.967065027Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871830966, 'readable_date': 
'2025-05-21 23:57:10', 'data': [{'key': 'ChargeRateMilePerHour', 'value': {'doubleValue': 13.511632653061701}}], 
'meta': {}, 'createdAt': '2025-05-21T23:57:10.966503912Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747871832467, 'readable_date': '2025-05-21 23:57:12', 'data': [{'key': 'EnergyRemaining', 'value': {'stringValue':
'39.519999116659164'}}], 'meta': {}, 'createdAt': '2025-05-21T23:57:12.467296598Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747871833467, 'readable_date': '2025-05-21 23:57:13', 'data': [{'key': 'BatteryLevel', 'value': 
{'stringValue': '60.41009463722398'}}], 'meta': {}, 'createdAt': '2025-05-21T23:57:13.467301666Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747871835966, 'readable_date': '2025-05-21 23:57:15', 'data': [{'key': 
'PackVoltage', 'value': {'stringValue': '374.629991626367'}}, {'key': 'PackCurrent', 'value': {'stringValue': 
'8.800000131130219'}}], 'meta': {}, 'createdAt': '2025-05-21T23:57:15.966962235Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747871837467, 'readable_date': '2025-05-21 23:57:17', 'data': [{'key': 'IdealBatteryRange', 'value': 
{'stringValue': '154.5306087908696'}}], 'meta': {}, 'createdAt': '2025-05-21T23:57:17.467427528Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747871856466, 'readable_date': '2025-05-21 23:57:36', 'data': [{'key': 
'ACChargingPower', 'value': {'stringValue': '3.500000052154064'}}], 'meta': {}, 'createdAt': 
'2025-05-21T23:57:36.466489652Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871856967, 'readable_date': 
'2025-05-21 23:57:36', 'data': [{'key': 'DCChargingEnergyIn', 'value': {'stringValue': '17.23999961465597'}}], 
'meta': {}, 'createdAt': '2025-05-21T23:57:36.967334878Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747871857467, 'readable_date': '2025-05-21 23:57:37', 'data': [{'key': 'Soc', 'value': {'stringValue': 
'59.74763406940063'}}], 'meta': {}, 'createdAt': '2025-05-21T23:57:37.467219471Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747871857967, 'readable_date': '2025-05-21 23:57:37', 'data': [{'key': 'ACChargingEnergyIn', 'value':
{'stringValue': '19.757500294414392'}}], 'meta': {}, 'createdAt': '2025-05-21T23:57:37.967317682Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747871860966, 'readable_date': '2025-05-21 23:57:40', 'data': [{'key': 
'ChargeRateMilePerHour', 'value': {'doubleValue': 13.094285714286272}}], 'meta': {}, 'createdAt': 
'2025-05-21T23:57:40.966696644Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871862467, 'readable_date': 
'2025-05-21 23:57:42', 'data': [{'key': 'EnergyRemaining', 'value': {'stringValue': '39.53999911621213'}}], 'meta':
{}, 'createdAt': '2025-05-21T23:57:42.467154285Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871863467, 
'readable_date': '2025-05-21 23:57:43', 'data': [{'key': 'BatteryLevel', 'value': {'stringValue': 
'60.47318611987381'}}], 'meta': {}, 'createdAt': '2025-05-21T23:57:43.467631033Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747871865966, 'readable_date': '2025-05-21 23:57:45', 'data': [{'key': 'PackVoltage', 'value': 
{'stringValue': '374.68999162502587'}}, {'key': 'PackCurrent', 'value': {'stringValue': '8.600000128149986'}}], 
'meta': {}, 'createdAt': '2025-05-21T23:57:45.966458211Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747871886467, 'readable_date': '2025-05-21 23:58:06', 'data': [{'key': 'ACChargingPower', 'value': {'stringValue':
'3.6000000536441803'}}], 'meta': {}, 'createdAt': '2025-05-21T23:58:06.467433344Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747871886966, 'readable_date': '2025-05-21 23:58:06', 'data': [{'key': 'DCChargingEnergyIn', 'value':
{'stringValue': '17.279999613761902'}}], 'meta': {}, 'createdAt': '2025-05-21T23:58:06.966990054Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747871887467, 'readable_date': '2025-05-21 23:58:07', 'data': [{'key': 'Soc', 
'value': {'stringValue': '59.84227129337539'}}], 'meta': {}, 'createdAt': '2025-05-21T23:58:07.467142712Z'}, 
{'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871887967, 'readable_date': '2025-05-21 23:58:07', 'data': [{'key': 
'ACChargingEnergyIn', 'value': {'stringValue': '19.78672251707207'}}], 'meta': {}, 'createdAt': 
'2025-05-21T23:58:07.967202537Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871890966, 'readable_date': 
'2025-05-21 23:58:10', 'data': [{'key': 'ChargeRateMilePerHour', 'value': {'doubleValue': 13.351836734694338}}], 
'meta': {}, 'createdAt': '2025-05-21T23:58:10.966491870Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747871892466, 'readable_date': '2025-05-21 23:58:12', 'data': [{'key': 'EnergyRemaining', 'value': {'stringValue':
'39.53999911621213'}}], 'meta': {}, 'createdAt': '2025-05-21T23:58:12.466688875Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747871893467, 'readable_date': '2025-05-21 23:58:13', 'data': [{'key': 'BatteryLevel', 'value': 
{'stringValue': '60.53627760252366'}}], 'meta': {}, 'createdAt': '2025-05-21T23:58:13.467211880Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747871895967, 'readable_date': '2025-05-21 23:58:15', 'data': [{'key': 
'PackCurrent', 'value': {'stringValue': '8.700000129640102'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'375.04999161697924'}}], 'meta': {}, 'createdAt': '2025-05-21T23:58:15.967426534Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747871897467, 'readable_date': '2025-05-21 23:58:17', 'data': [{'key': 'IdealBatteryRange', 'value': 
{'stringValue': '154.857139395816'}}], 'meta': {}, 'createdAt': '2025-05-21T23:58:17.467656379Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747871916967, 'readable_date': '2025-05-21 23:58:36', 'data': [{'key': 
'DCChargingEnergyIn', 'value': {'stringValue': '17.319999612867832'}}], 'meta': {}, 'createdAt': 
'2025-05-21T23:58:36.967293943Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871917467, 'readable_date': 
'2025-05-21 23:58:37', 'data': [{'key': 'Soc', 'value': {'stringValue': '59.873817034700316'}}], 'meta': {}, 
'createdAt': '2025-05-21T23:58:37.467528774Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871917966, 
'readable_date': '2025-05-21 23:58:37', 'data': [{'key': 'ACChargingEnergyIn', 'value': {'stringValue': 
'19.81672251751911'}}], 'meta': {}, 'createdAt': '2025-05-21T23:58:37.966937942Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747871920967, 'readable_date': '2025-05-21 23:58:40', 'data': [{'key': 'ChargeRateMilePerHour', 
'value': {'doubleValue': 13.516530612245534}}], 'meta': {}, 'createdAt': '2025-05-21T23:58:40.967535733Z'}, {'vin':
'5YJ3E7EB1KF334219', 'timestamp': 1747871922467, 'readable_date': '2025-05-21 23:58:42', 'data': [{'key': 
'EnergyRemaining', 'value': {'stringValue': '39.559999115765095'}}], 'meta': {}, 'createdAt': 
'2025-05-21T23:58:42.467517078Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871925966, 'readable_date': 
'2025-05-21 23:58:45', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': '374.9499916192144'}}, {'key': 
'PackCurrent', 'value': {'stringValue': '8.900000132620335'}}], 'meta': {}, 'createdAt': 
'2025-05-21T23:58:45.966962982Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871938467, 'readable_date': 
'2025-05-21 23:58:58', 'data': [{'key': 'ACChargingPower', 'value': {'stringValue': '3.500000052154064'}}], 'meta':
{}, 'createdAt': '2025-05-21T23:58:58.467682086Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871946967, 
'readable_date': '2025-05-21 23:59:06', 'data': [{'key': 'DCChargingEnergyIn', 'value': {'stringValue': 
'17.339999612420797'}}], 'meta': {}, 'createdAt': '2025-05-21T23:59:06.967264622Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747871947467, 'readable_date': '2025-05-21 23:59:07', 'data': [{'key': 'Soc', 'value': 
{'stringValue': '59.90536277602524'}}, {'key': 'BatteryLevel', 'value': {'stringValue': '60.59936908517351'}}], 
'meta': {}, 'createdAt': '2025-05-21T23:59:07.467389080Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747871947967, 'readable_date': '2025-05-21 23:59:07', 'data': [{'key': 'ACChargingEnergyIn', 'value': 
{'stringValue': '19.846444740184236'}}], 'meta': {}, 'createdAt': '2025-05-21T23:59:07.967451420Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747871950967, 'readable_date': '2025-05-21 23:59:10', 'data': [{'key': 
'ChargeRateMilePerHour', 'value': {'doubleValue': 13.171632653061934}}], 'meta': {}, 'createdAt': 
'2025-05-21T23:59:10.967380713Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871952466, 'readable_date': 
'2025-05-21 23:59:12', 'data': [{'key': 'EnergyRemaining', 'value': {'stringValue': '39.61999911442399'}}], 'meta':
{}, 'createdAt': '2025-05-21T23:59:12.466775936Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871955967, 
'readable_date': '2025-05-21 23:59:15', 'data': [{'key': 'PackVoltage', 'value': {'stringValue': 
'374.87999162077904'}}, {'key': 'PackCurrent', 'value': {'stringValue': '8.600000128149986'}}], 'meta': {}, 
'createdAt': '2025-05-21T23:59:15.967201359Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871957467, 
'readable_date': '2025-05-21 23:59:17', 'data': [{'key': 'IdealBatteryRange', 'value': {'stringValue': 
'155.02040469828918'}}], 'meta': {}, 'createdAt': '2025-05-21T23:59:17.467117160Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747871968466, 'readable_date': '2025-05-21 23:59:28', 'data': [{'key': 'ACChargingPower', 'value': 
{'stringValue': '3.6000000536441803'}}], 'meta': {}, 'createdAt': '2025-05-21T23:59:28.466942731Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747871977467, 'readable_date': '2025-05-21 23:59:37', 'data': [{'key': 
'BatteryLevel', 'value': {'stringValue': '60.66246056782334'}}], 'meta': {}, 'createdAt': 
'2025-05-21T23:59:37.467087172Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871977967, 'readable_date': 
'2025-05-21 23:59:37', 'data': [{'key': 'ACChargingEnergyIn', 'value': {'stringValue': '19.875889185067447'}}], 
'meta': {}, 'createdAt': '2025-05-21T23:59:37.967104287Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 
1747871980967, 'readable_date': '2025-05-21 23:59:40', 'data': [{'key': 'ChargeRateMilePerHour', 'value': 
{'doubleValue': 13.413061224490443}}, {'key': 'DCChargingEnergyIn', 'value': {'stringValue': 
'17.359999611973763'}}], 'meta': {}, 'createdAt': '2025-05-21T23:59:40.967294299Z'}, {'vin': '5YJ3E7EB1KF334219', 
'timestamp': 1747871981466, 'readable_date': '2025-05-21 23:59:41', 'data': [{'key': 'Soc', 'value': 
{'stringValue': '59.93690851735016'}}], 'meta': {}, 'createdAt': '2025-05-21T23:59:41.466575740Z'}, {'vin': 
'5YJ3E7EB1KF334219', 'timestamp': 1747871982467, 'readable_date': '2025-05-21 23:59:42', 'data': [{'key': 
'EnergyRemaining', 'value': {'stringValue': '39.639999113976955'}}], 'meta': {}, 'createdAt': 
'2025-05-21T23:59:42.467501034Z'}, {'vin': '5YJ3E7EB1KF334219', 'timestamp': 1747871985967, 'readable_date': 
'2025-05-21 23:59:45', 'data': [{'key': 'ChargeAmps', 'value': {'stringValue': '16.00000023841858'}}, {'key': 
'BmsFullchargecomplete', 'value': {'stringValue': 'false'}}, {'key': 'PackVoltage', 'value': {'stringValue': 
'375.10999161563814'}}, {'key': 'PackCurrent', 'value': {'stringValue': '8.800000131130219'}}, {'key': 
'ChargeState', 'value': {'stringValue': 'Enable'}}], 'meta': {}, 'createdAt': '2025-05-21T23:59:45.967816716Z'}]

In [ ]:
test_rep_bis = [{"vin": "5YJ3E7EB1KF334219", "timestamp": 1748318343277, "readable_date": "2025-05-27 03:59:03", "data": [{"key": "InsideTemp", "value": {"stringValue": "19.100000880658627"}}, {"key": "FastChargerType", "value": {"invalid": True}}, {"key": "PackCurrent", "value": {"stringValue": "0"}}, {"key": "ModuleTempMax", "value": {"stringValue": "26"}}, {"key": "ChargeCurrentRequest", "value": {"stringValue": "16"}}, {"key": "ChargePortColdWeatherMode", "value": {"stringValue": "False"}}, {"key": "DCChargingEnergyIn", "value": {"stringValue": "17.419999610632658"}}, {"key": "DCDCEnable", "value": {"stringValue": "False"}}, {"key": "ChargerPhases", "value": {"invalid": True}}, {"key": "PreconditioningEnabled", "value": {"stringValue": "False"}}, {"key": "ChargeCurrentRequestMax", "value": {"stringValue": "16"}}, {"key": "BatteryLevel", "value": {"stringValue": "49.2429022082019"}}, {"key": "ACChargingPower", "value": {"stringValue": "0"}}, {"key": "EstBatteryRange", "value": {"stringValue": "113.16217657792016"}}, {"key": "BmsFullchargecomplete", "value": {"stringValue": "False"}}, {"key": "ChargeAmps", "value": {"stringValue": "0"}}, {"key": "LifetimeEnergyUsed", "value": {"stringValue": "26604.12326362799"}}, {"key": "HvacACEnabled", "value": {"booleanValue": False}}, {"key": "BatteryHeaterOn", "value": {"stringValue": "False"}}, {"key": "IsolationResistance", "value": {"stringValue": "4280"}}, {"key": "DetailedChargeState", "value": {"detailedChargeStateValue": "DetailedChargeStateDisconnected"}}, {"key": "BrickVoltageMin", "value": {"stringValue": "3.836000182200223"}}, {"key": "BrickVoltageMax", "value": {"stringValue": "3.838000182295218"}}, {"key": "EstimatedHoursToChargeTermination", "value": {"invalid": True}}, {"key": "ChargePort", "value": {"stringValue": "CCS"}}, {"key": "ChargeState", "value": {"stringValue": "Idle"}}, {"key": "HvacPower", "value": {"hvacPowerValue": "HvacPowerStateOff"}}, {"key": "EfficiencyPackage", "value": {"stringValue": "Default"}}, {"key": "HvacAutoMode", "value": {"hvacAutoModeValue": "HvacAutoModeStateOn"}}, {"key": "SentryMode", "value": {"stringValue": "Off"}}, {"key": "HvacFanSpeed", "value": {"intValue": 2}}, {"key": "BMSState", "value": {"stringValue": "Standby"}}, {"key": "FastChargerPresent", "value": {"stringValue": "False"}}, {"key": "ModuleTempMin", "value": {"stringValue": "24.5"}}, {"key": "Odometer", "value": {"stringValue": "64016.35690902214"}}, {"key": "Soc", "value": {"stringValue": "48.958990536277604"}}, {"key": "DefrostMode", "value": {"defrostModeValue": "DefrostModeStateOff"}}, {"key": "ChargeEnableRequest", "value": {"stringValue": "True"}}, {"key": "ACChargingEnergyIn", "value": {"stringValue": "17.83427804352963"}}, {"key": "ChargeRateMilePerHour", "value": {"doubleValue": 0}}, {"key": "ChargingCableType", "value": {"invalid": True}}, {"key": "VehicleSpeed", "value": {"invalid": True}}, {"key": "OutsideTemp", "value": {"stringValue": "12"}}, {"key": "RatedRange", "value": {"stringValue": "126.69387471919158"}}, {"key": "EuropeVehicle", "value": {"booleanValue": True}}, {"key": "PackVoltage", "value": {"stringValue": "368.1199917718768"}}, {"key": "IdealBatteryRange", "value": {"stringValue": "126.69387471919158"}}, {"key": "ClimateKeeperMode", "value": {"climateKeeperModeValue": "ClimateKeeperModeStateOff"}}, {"key": "RearDefrostEnabled", "value": {"booleanValue": False}}, {"key": "CarType", "value": {"stringValue": "Model3"}}, {"key": "DefrostForPreconditioning", "value": {"booleanValue": False}}, {"key": "EnergyRemaining", "value": {"stringValue": "32.65999926999211"}}, {"key": "ChargeLimitSoc", "value": {"stringValue": "62"}}, {"key": "DCChargingPower", "value": {"stringValue": "0"}}], "meta": {}, "createdAt": "2025-05-27T03:59:03.277232687Z"}, {"vin": "5YJ3E7EB1KF334219", "timestamp": 1748318344277, "readable_date": "2025-05-27 03:59:04", "data": [{"key": "ChargerVoltage", "value": {"doubleValue": 1.716}}], "meta": {}, "createdAt": "2025-05-27T03:59:04.277224934Z"}]

In [ ]:
explode_data_spark(test_rep_bis).show()

In [ ]:
res = get_raw_tss_from_keys_spark(spark.createDataFrame(line_test), s3)

In [ ]:
res.show()

In [ ]:
spark.createDataFrame(line_test).show()

In [ ]:
response.show()

In [ ]:
get_raw_tss_from_keys_spark(spark.createDataFrame(line_test), s3)

In [ ]:
test = get_raw_tss_from_keys_spark(response, s3)

In [ ]:
test.toPandas()

In [ ]:
response.toPandas()

# pass

In [ ]:
raw_tss = []
grouped = keys.groupby(pd.Grouper(key='date', freq='W-MON'))
grouped_items = list(grouped)[:1]
for week, week_keys in track(grouped_items, description="Processing weekly groups"):
    week_date = week.date().strftime('%Y-%m-%d')
    logger.debug(f"Parsing the responses of the week {week_date}:")
    logger.debug(f"{len(week_keys)} keys to parse for {week_keys['vin'].nunique()} vins.")
    logger.debug(f"This represents {round(len(week_keys) / len(keys) * 100)}% of the total keys to parse.")
    responses = s3.read_multiple_json_files(week_keys["key"].tolist(), max_workers=64)
    logger.debug(f"Read the responses.")
    with ThreadPoolExecutor(max_workers=64) as executor:
        week_raw_tss = list(executor.map(DF.from_records, responses))
    logger.debug(f"Parsed the responses:")
    week_raw_tss = pd.concat([explode_data(df) for df in week_raw_tss])
    logger.debug(f"Concatenated the responses into a single DF.")
    raw_tss.append(week_raw_tss)
    logger.debug("")

In [ ]:
raw_tss

### Processed tss

In [ ]:
from transform.processed_tss.ProcessedTimeSeries import ProcessedTimeSeries

In [ ]:
def compute_charge_n_discharge_masks(tss:DF) -> DF:
        # We use a nullable boolean Series to represnet the rows where:
        tss["nan_charging"] = (
            Series(pd.NA, index=tss.index, dtype="boolean")# We are not sure of anything.
            .mask(tss["charging_status"].isin(IN_CHARGE_CHARGING_STATUS_VALS), True)# We are sure that the vehicle is in charge.
            .mask(tss["charging_status"].isin(IN_DISCHARGE_CHARGING_STATUS_VALS), False)# We are sure that the vehicle is not in charge.
        )
        # If a period of uncertainty (NaN) is surrounded by equal periods of certainties (True-NaN-True or False-NaN-False),
        # We will fill them to the value of these certainties.
        # However there are edge cases that have multiple days of uncertainties periods (I can't find the VIN but I'm sure you can ;-) )
        # Interestingly enough the charge_energy_added variable does not get forwared that far and gets reset to zero. 
        # This would create outliers in our charge SoH estimation as we estimate the energy_gained as the diff between the last(0) and first value of charge_energy_added.
        # So we set a maximal uncertainty period duration over which we don't fill it.
        tss["nan_date"] = tss["date"].mask(tss["nan_charging"].isna())
        tss[["ffill_charging", "ffill_date"]] = tss.groupby("vin", observed=True)[["nan_charging", "nan_date"]].ffill()
        tss[["bfill_charging", "bfill_date"]] = tss.groupby("vin", observed=True)[["nan_charging", "nan_date"]].bfill()
        nan_period_duration:Series = tss.eval("bfill_date - ffill_date")
        fill_unknown_period = tss.eval("ffill_charging.eq(bfill_charging) & @nan_period_duration.le(@MAX_CHARGE_TD)")
        tss["nan_charging"] = tss["nan_charging"].mask(fill_unknown_period, tss["ffill_charging"])
        # As mentioned before, the SoC oscillates at [charge_limit_soc - ~3%, charge_limit_soc] so we set these periods to NaN as well.
        tss["nan_charging"] = tss["nan_charging"].mask(tss["soc"] >= (tss["charge_limit_soc"] - 3))
        # Then we seperate the Series into two, more explicit, columns.
        tss["in_charge"] = tss.eval("nan_charging.notna() & nan_charging")
        tss["in_discharge"] = tss.eval("nan_charging.notna() & ~nan_charging")
        return tss.drop(columns=["nan_charging", "ffill_charging", "bfill_charging", "ffill_date", "bfill_date"])
    
def compute_enenergy_added(tss:DF) -> DF:
        tss['charge_energy_added'] = tss['dc_charge_energy_added'].where(
            tss['dc_charge_energy_added'].notnull() & 
            (tss['dc_charge_energy_added'] > 0), 
            tss['ac_charge_energy_added'])
        return tss
    
def compute_charge_idx_bis(tss):

        tss = tss.pipe(compute_enenergy_added)
        tss_na = tss.dropna(subset=['soc']).copy()
        tss_na['soc_diff'] = tss_na.groupby('vin', observed=True)['soc'].diff()
        tss_na['soc_diff_rolling'] = tss_na['soc_diff'].rolling(window=5, min_periods=1).mean()
        # Determine trend
        tss_na['trend'] = tss_na['soc_diff_rolling'].apply(lambda x: 1 if x > 0 else -1 if x < 0 else np.nan)
        tss_na['trend'] = tss_na['trend'].ffill()

        def detect_trend_change(group):

            group['prev_trend'] = group['trend'].shift(1)
            group['prev_prev_trend'] = group['trend'].shift(2)
            
            group['prev_date'] = group['date'].shift(1)
            group['time_diff_min'] = (group['date'] - group['prev_date']).dt.total_seconds() / 60
            group['time_gap'] = group['time_diff_min'] > 60  

            # Faire une sépration charge_idx et discharge_idx
            group['trend_change'] = (
                (((group['trend'] != group['prev_trend']) & 
                  (group['prev_trend'] == group['prev_prev_trend']) ) |
                group['time_gap'])
            )
            group.loc[group.index[0:2], 'trend_change'] = False
            return group


        tss_na = tss_na.groupby('vin', observed=True).apply(detect_trend_change).reset_index(drop=True)
        
        # Compute charge id
        tss_na['in_charge_idx'] = tss_na.groupby('vin',  observed=True)['trend_change'].cumsum()
        tss = tss.merge(tss_na[["soc", "date", "vin", 'soc_diff', 'in_charge_idx', 'trend', 'prev_trend', 'prev_prev_trend', 'trend_change',]], 
                        on=["soc", "date", "vin"], how="left")
        tss[["odometer","in_charge_idx"]] = tss[["odometer", "in_charge_idx"]].ffill()
        return tss

In [ ]:
from core.pandas_utils import safe_locate, safe_astype, str_lower_columns
from transform.fleet_info.main import fleet_info 

In [ ]:
make = "tesla-fleet-telemetry"
def run(tss):
    tss = tss.rename(columns=RENAME_COLS_DICT, errors="ignore")
    tss = tss.pipe(safe_locate, col_loc=list(COL_DTYPES.keys()), logger=logger)
    tss = tss.pipe(safe_astype, COL_DTYPES, logger=logger)
    tss = tss.pipe(normalize_units_to_metric)
    tss = tss.sort_values(by=["vin", "date"])
    tss = tss.pipe(str_lower_columns, COLS_TO_STR_LOWER)
    tss = tss.pipe(compute_date_vars)
    tss = tss.pipe(compute_charge_n_discharge_vars)
    tss = tss.merge(fleet_info, on="vin", how="left")
    tss = tss.eval("age = date.dt.tz_localize(None) - start_date.dt.tz_localize(None)")
    # It seems that the reset_index calls doesn't reset the "vin" into a category if the groupby's by argument was categorical.
    # So we recall astype on the "vin"  in case it is supposed to be categorical.
    tss = tss.astype({"vin": COL_DTYPES["vin"]})
    return tss

def compute_charge_n_discharge_vars(tss:DF) -> DF:
    return (
        tss
        # Compute the in_charge and in_discharge masks 
        .pipe(compute_charge_n_discharge_masks, IN_CHARGE_CHARGING_STATUS_VALS, IN_DISCHARGE_CHARGING_STATUS_VALS)
        # Compute the correspding indices to perfrom split-apply-combine ops
        .pipe(compute_idx_from_masks, ["in_charge", "in_discharge"])
        # We recompute the masks by trimming off the points that have the first and last soc values
        # This is done to reduce the noise in the output due to measurments noise.
        .pipe(trim_leading_n_trailing_soc_off_masks, ["in_charge", "in_discharge"]) 
        .pipe(compute_idx_from_masks, ["trimmed_in_charge", "trimmed_in_discharge"])
        .pipe(compute_cum_var, "power", "cum_energy")
        .pipe(compute_cum_var, "charger_power", "cum_charge_energy_added")
        .pipe(compute_status_col)
    )

def normalize_units_to_metric( tss:DF) -> DF:
    tss["odometer"] = tss["odometer"] * ODOMETER_MILES_TO_KM.get(make, 1)
    return tss
from scipy.integrate import cumulative_trapezoid
def compute_cum_var( tss: DF, var_col:str, cum_var_col:str) -> DF:
    if not var_col in tss.columns:
        logger.debug(f"{var_col} not found, not computing {cum_var_col}.")
        return tss
    logger.debug(f"Computing {cum_var_col} from {var_col}.")
    tss[cum_var_col] = (
        cumulative_trapezoid(
            # Leave the keywords as default order is y x not x y (-_-)
            # Make sure that date time units are in seconds before converting to int
            x=tss["date"].dt.as_unit("s").astype(int),
            y=tss[var_col].fillna(0).values,
            initial=0,
        )            
        .astype("float32")
    )
    tss[cum_var_col] *= KJ_TO_KWH # Convert from kj to kwh
    # Reset value to zero at the start of each vehicle time series
    # This is better than performing a groupby.apply with cumulative_trapezoid
    tss[cum_var_col] -= tss.groupby("vin", observed=True)[cum_var_col].transform("first")
    return tss

def compute_date_vars( tss:DF) -> DF:
    tss["time_diff"] = tss.groupby("vin", observed=False)["date"].diff()
    tss["sec_time_diff"] = tss["time_diff"].dt.total_seconds()
    return tss

def compute_charge_n_discharge_masks(tss:DF, in_charge_vals:list, in_discharge_vals:list) -> DF:
    """Computes the `in_charge` and `in_discharge` masks either from the charging_status column or from the evolution of the soc over time."""
    if make in CHARGE_MASK_WITH_CHARGING_STATUS_MAKES:
        return charge_n_discharging_masks_from_charging_status(tss, in_charge_vals, in_discharge_vals)
    if make in CHARGE_MASK_WITH_SOC_DIFFS_MAKES:
        return charge_n_discharging_masks_from_soc_diff(tss)
    raise ValueError(MAKE_NOT_SUPPORTED_ERROR.format(make=make))

def charge_n_discharging_masks_from_soc_diff( tss:DF) -> DF:
    tss_grp = tss.groupby("vin", observed=True)
    tss["soc_ffilled"] = tss_grp["soc"].ffill()
    tss["soc_diff"] = tss_grp["soc_ffilled"].diff()
    tss["soc_diff"] /= tss["soc_diff"].abs()
    soc_diff_ffilled = tss_grp["soc_diff"].ffill()
    soc_diff_bfilled = tss_grp["soc_diff"].bfill()
    tss["in_charge"] = soc_diff_ffilled.gt(0, fill_value=False) & soc_diff_bfilled.gt(0, fill_value=False)
    tss["in_discharge"] = soc_diff_ffilled.lt(0, fill_value=False) & soc_diff_bfilled.lt(0, fill_value=False)
    return tss

def charge_n_discharging_masks_from_charging_status( tss:DF, in_charge_vals:list, in_discharge_vals:list) -> DF:
    assert "charging_status" in tss.columns, NO_CHARGING_STATUS_COL_ERROR
    return (
        tss
        .eval(f"in_charge = charging_status in {in_charge_vals}")
        .eval(f"in_discharge = charging_status in {in_discharge_vals}")
    )

def trim_leading_n_trailing_soc_off_masks( tss:DF, masks:list[str]) -> DF:
    for mask in masks:
        tss["naned_soc"] = tss["soc"].where(tss[mask])
        soc_grp = tss.groupby(["vin", mask + "_idx"], observed=True)["naned_soc"]
        trailing_soc = soc_grp.transform("first")
        leading_soc = soc_grp.transform("last")
        tss["trailing_soc"] = trailing_soc
        tss["leading_soc"] = leading_soc
        tss[f"trimmed_{mask}"] = tss[mask] & (tss["soc"] != trailing_soc) & (tss["soc"] != leading_soc)
    tss = tss.drop(columns="naned_soc")
    return tss
max_td = TD(hours=1, minutes=30)
def compute_idx_from_masks( tss: DF, masks:list[str]) -> DF:
    for mask in masks:
        idx_col_name = f"{mask}_idx"
        shifted_mask = tss.groupby("vin", observed=True)[mask].shift(fill_value=False)
        tss["new_period_start_mask"] = shifted_mask.ne(tss[mask]) 
        if max_td is not None:
            tss["new_period_start_mask"] |= (tss["time_diff"] > max_td)
        tss[idx_col_name] = tss.groupby("vin", observed=True)["new_period_start_mask"].cumsum().astype("uint16")
        tss.drop(columns=["new_period_start_mask"], inplace=True)
    return tss

def compute_status_col( tss:DF) -> DF:
    tss_grp = tss.groupby("vin", observed=True)
    status = tss["in_charge"].map({True: "charging", False:"discharging", pd.NA:"unknown"})
    tss["status"] = status.mask(
        tss["in_charge"].eq(False, fill_value=True),
        np.where(tss_grp["odometer"].diff() > 0, "moving", "idle_discharging"),
    )
    return tss

In [ ]:
processed_tss = run(raw_tss)

In [ ]:

class TeslaProcessedTimeSeries(ProcessedTimeSeries):

    def __init__(self, make:str="tesla", id_col:str="vin", log_level:str="INFO", max_td:TD=MAX_TD, force_update:bool=False, **kwargs):
        self.logger = getLogger(make)
        set_level_of_loggers_with_prefix(log_level, make)
        super().__init__(make, id_col, log_level, max_td, force_update, **kwargs)

    def compute_charge_n_discharge_vars(self, tss:DF) -> DF:
        return (
            tss
            .pipe(self.compute_charge_n_discharge_masks)
            .pipe(self.compute_charge_idx_bis)
            # .pipe(self.compute_idx_from_masks, ["in_charge"])
            # .pipe(self.trim_leading_n_trailing_soc_off_masks, ["in_charge", "in_discharge"])
            # # .pipe(self.compute_idx_from_masks, ["trimmed_in_charge", "trimmed_in_discharge"])
        )

    def compute_charge_n_discharge_masks(self, tss:DF) -> DF:
        self.logger.debug("Computing tesla specific charge and discharge masks")
        # We use a nullable boolean Series to represnet the rows where:
        tss["nan_charging"] = (
            Series(pd.NA, index=tss.index, dtype="boolean")# We are not sure of anything.
            .mask(tss["charging_status"].isin(IN_CHARGE_CHARGING_STATUS_VALS), True)# We are sure that the vehicle is in charge.
            .mask(tss["charging_status"].isin(IN_DISCHARGE_CHARGING_STATUS_VALS), False)# We are sure that the vehicle is not in charge.
        )
        # If a period of uncertainty (NaN) is surrounded by equal periods of certainties (True-NaN-True or False-NaN-False),
        # We will fill them to the value of these certainties.
        # However there are edge cases that have multiple days of uncertainties periods (I can't find the VIN but I'm sure you can ;-) )
        # Interestingly enough the charge_energy_added variable does not get forwared that far and gets reset to zero. 
        # This would create outliers in our charge SoH estimation as we estimate the energy_gained as the diff between the last(0) and first value of charge_energy_added.
        # So we set a maximal uncertainty period duration over which we don't fill it.
        tss["nan_date"] = tss["date"].mask(tss["nan_charging"].isna())
        tss[["ffill_charging", "ffill_date"]] = tss.groupby("vin", observed=True)[["nan_charging", "nan_date"]].ffill()
        tss[["bfill_charging", "bfill_date"]] = tss.groupby("vin", observed=True)[["nan_charging", "nan_date"]].bfill()
        nan_period_duration:Series = tss.eval("bfill_date - ffill_date")
        fill_unknown_period = tss.eval("ffill_charging.eq(bfill_charging) & @nan_period_duration.le(@MAX_CHARGE_TD)")
        tss["nan_charging"] = tss["nan_charging"].mask(fill_unknown_period, tss["ffill_charging"])
        # As mentioned before, the SoC oscillates at [charge_limit_soc - ~3%, charge_limit_soc] so we set these periods to NaN as well.
        tss["nan_charging"] = tss["nan_charging"].mask(tss["soc"] >= (tss["charge_limit_soc"] - 3))
        # Then we seperate the Series into two, more explicit, columns.
        tss["in_charge"] = tss.eval("nan_charging.notna() & nan_charging")
        tss["in_discharge"] = tss.eval("nan_charging.notna() & ~nan_charging")
        return tss.drop(columns=["nan_charging", "ffill_charging", "bfill_charging", "ffill_date", "bfill_date"])
    
    def compute_charge_n_discharge_masks_bis(self, tss:DF) -> DF:
        self.logger.debug("Computing tesla specific charge and discharge masks")

        tss_na = tss.dropna(subset=['soc']).copy()

        tss_na['soc_diff'] = tss_na.groupby('vin', observed=True)['soc'].diff()

        tss_na['trend'] = tss_na['soc_diff'].apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0)

        #tss_na['trend_change'] = tss_na.groupby('vin', observed=True)['trend'].transform(lambda x: x != x.shift())
        tss = tss.merge(tss_na[["soc", "date", "vin", 'soc_diff', 'trend']], 
                        on=["soc", "date", "vin"], how="left")
        tss[["trend", "soc", "odometer",]].bfill(inplace=True)
        tss["in_charge"] = tss.eval('trend==1')
        tss["in_discharge"] = tss.eval('trend==-1')
        return tss
    
    
    def compute_enenergy_added(self, tss:DF) -> DF:
        tss['charge_energy_added'] = tss['dc_charge_energy_added'].where(
            tss['dc_charge_energy_added'].notnull() & 
            (tss['dc_charge_energy_added'] > 0), 
            tss['ac_charge_energy_added'])
        return tss
    
    # def compute_charge_idx(self, tss:DF) -> DF:
    #     self.logger.debug("Computing tesla specific charge index.")
    #     if self.make == 'tesla-fleet-telemetry':
    #         tss = tss.pipe(self.compute_enenergy_added)
    #     tss_grp = tss.groupby("vin", observed=False)
    #     tss["charge_energy_added"] = tss_grp["charge_energy_added"].ffill()
    #     energy_added_over_time = tss_grp['charge_energy_added'].diff().div(tss["sec_time_diff"].values)
    #     # charge_energy_added is cummulative and forward filled, 
    #     # We check that the charge_energy_added decreases too fast to make sure that  correctly indentify two charging periods before and after a gap as two separate charging periods.
    #     new_charge_mask = energy_added_over_time.lt(MIN_POWER_LOSS, fill_value=0) 
    #     # For the same reason, we ensure that there are no gaps bigger than MAX_CHARGE_TD in between to rows of the same charging period.
    #     new_charge_mask |= tss["time_diff"].gt(MAX_CHARGE_TD) 
    #     # And of course we also check that there is no change of status. 
    #     new_charge_mask |= (~tss_grp["in_charge"].shift().bfill() & tss["in_charge"]) 
    #     tss["in_charge_idx"] = new_charge_mask.groupby(tss["vin"], observed=True).cumsum()
    #     print(tss["in_charge_idx"].count() / len(tss))
    #     tss["in_charge_idx"] = tss["in_charge_idx"].fillna(-1).astype("uint16")
    #     return tss
    
    def compute_charge_idx_bis(self, tss):

        if self.make == 'tesla-fleet-telemetry':
                    tss = tss.pipe(self.compute_enenergy_added)
        tss_na = tss.dropna(subset=['soc']).copy()
        tss_na['soc_diff'] = tss_na.groupby('vin', observed=True)['soc'].diff()
        tss_na['soc_diff_rolling'] = tss_na['soc_diff'].rolling(window=5, min_periods=1).mean()
        # Determine trend
        tss_na['trend'] = tss_na['soc_diff_rolling'].apply(lambda x: 1 if x > 0 else -1 if x < 0 else np.nan)
        tss_na['trend'] = tss_na['trend'].ffill()

        def detect_trend_change(group):

            group['prev_trend'] = group['trend'].shift(1)
            group['prev_prev_trend'] = group['trend'].shift(2)
            
            group['prev_date'] = group['date'].shift(1)
            group['time_diff_min'] = (group['date'] - group['prev_date']).dt.total_seconds() / 60
            group['time_gap'] = group['time_diff_min'] > 60  

            # Faire une sépration charge_idx et discharge_idx
            group['trend_change'] = (
                (((group['trend'] != group['prev_trend']) & 
                  (group['prev_trend'] == group['prev_prev_trend']) ) |
                group['time_gap'])
            )
            group.loc[group.index[0:2], 'trend_change'] = False
            return group


        tss_na = tss_na.groupby('vin', observed=True).apply(detect_trend_change).reset_index(drop=True)
        
        # Compute charge id
        tss_na['in_charge_idx'] = tss_na.groupby('vin',  observed=True)['trend_change'].cumsum()
        tss = tss.merge(tss_na[["soc", "date", "vin", 'soc_diff', 'in_charge_idx', 'trend', 'prev_trend', 'prev_prev_trend', 'trend_change',]], 
                        on=["soc", "date", "vin"], how="left")
        tss[["odometer","in_charge_idx"]] = tss[["odometer", "in_charge_idx"]].ffill()
        return tss


In [ ]:
processed_tss = TeslaProcessedTimeSeries("tesla-fleet-telemetry", force_update=True)
processed_tss['in_charge_idx'] = processed_tss['in_charge_idx'].astype(str)


### raw results

In [ ]:
from core.stats_utils import series_start_end_diff

In [ ]:
raw_results = (processed_tss.groupby(["vin", "in_charge_idx"], observed=True, as_index=False).agg(
            ac_energy_added_min=pd.NamedAgg("ac_charge_energy_added", "min"),
            dc_energy_added_min=pd.NamedAgg("dc_charge_energy_added", "min"),
            ac_energy_added_end=pd.NamedAgg("ac_charge_energy_added", "last"),
            dc_energy_added_end=pd.NamedAgg("dc_charge_energy_added", "last"),
            soc_diff=pd.NamedAgg("soc", series_start_end_diff),
            inside_temp=pd.NamedAgg("inside_temp", "mean"),
            net_capacity=pd.NamedAgg("net_capacity", "first"),
            range=pd.NamedAgg("range", "first"),
            odometer=pd.NamedAgg("odometer", "first"),
            version=pd.NamedAgg("version", "first"),
            size=pd.NamedAgg("soc", "size"),
            model=pd.NamedAgg("model", "first"),
            date=pd.NamedAgg("date", "first"),
            ac_charging_power=pd.NamedAgg("ac_charging_power", "median"),
            dc_charging_power=pd.NamedAgg("dc_charging_power", "median"),
            tesla_code=pd.NamedAgg("tesla_code", "first"),
        )
        .eval("charging_power = ac_charging_power + dc_charging_power")
        .eval("ac_energy_added = ac_energy_added_end  - ac_energy_added_min")
        .eval("dc_energy_added = dc_energy_added_end  - dc_energy_added_min")
        .assign(energy_added=lambda df: np.maximum(df["ac_energy_added"], df["dc_energy_added"]))
        .eval("soh = energy_added / (soc_diff / 100.0 * net_capacity)")
        .eval("level_1 = soc_diff * (charging_power < 8) / 100")
        .eval("level_2 = soc_diff * (charging_power.between(8, 45)) / 100")
        .eval("level_3 = soc_diff * (charging_power > 45) / 100")
        .sort_values(["tesla_code", "vin", "date"]))

In [ ]:
25.788389 / (40.029564 / 100 * 60)

In [ ]:
raw_results[(raw_results['vin']=="LRWYGCFS6PC992837")][['soh', 'odometer', 'soc_diff', "energy_added"]]

In [ ]:
# pour repartir de ce qui est stocké
#raw_results_origin = get_results_origin(force_update=False)

### Processed results

In [ ]:
from transform.processed_results.main import *

In [ ]:
SOH_FILTER_EVAL = {
     "tesla-fleet-telemetry-30": "soh = soh.where(soc_diff > 30 & soh.between(0.75, 1.05))",
     "tesla-fleet-telemetry-25": "soh = soh.where(soc_diff > 25 & soh.between(0.75, 1.05))",
     "tesla-fleet-telemetry-20": "soh = soh.where(soc_diff > 20 & soh.between(0.75, 1.05))",
     "tesla-fleet-telemetry-15": "soh = soh.where(soc_diff > 15 & soh.between(0.75, 1.05))",
     "tesla-fleet-telemetry-8": "soh = soh.where(soc_diff > 8 & soh.between(0.75, 1.05))",
}


In [ ]:
def get_processed_results(brand:str) -> DF:
    logger.info(f"{'Processing ' + brand + ' results.':=^{50}}")
    results =  (
        raw_results
        # Some raw estimations may have inf values, this will make mask_out_outliers_by_interquartile_range and force_monotonic_decrease fail
        # So we replace them by NaNs.
        .assign(soh=lambda df: df["soh"].replace([np.inf, -np.inf], np.nan))
        .sort_values(["vin", "date"])
        .pipe(make_charge_levels_presentable)
        .eval(SOH_FILTER_EVAL[brand])
        .pipe(agg_results_by_update_frequency)
        .groupby('vin', observed=True)
        .apply(make_soh_presentable_per_vehicle, include_groups=False)
        .reset_index(level=0)
        #.pipe(filter_results_by_lines_bounds, VALID_SOH_POINTS_LINE_BOUNDS, logger=logger)
        .sort_values(["vin", "date"])
    )
    results["soh"] = results.groupby("vin", observed=True)["soh"].ffill()
    results["soh"] = results.groupby("vin", observed=True)["soh"].bfill()
    results["odometer"] = results.groupby("vin", observed=True)["odometer"].ffill()
    results["odometer"] = results.groupby("vin", observed=True)["odometer"].bfill()
    return results

In [ ]:
processed_results_30 = get_processed_results('tesla-fleet-telemetry-30')
processed_results_25 = get_processed_results('tesla-fleet-telemetry-25')
processed_results_20 = get_processed_results('tesla-fleet-telemetry-20')
processed_results_15 = get_processed_results('tesla-fleet-telemetry-15')
processed_results_8 = get_processed_results('tesla-fleet-telemetry-8')

In [ ]:
px.scatter(raw_results[(raw_results['soh'] >.7) &(raw_results['soh'] < 1.05)].dropna(subset='soh'), x='odometer', y='soh', color='vin')

In [ ]:
px.scatter(processed_results_30[(processed_results_30['soh'] > .75) &(processed_results_30['soh'] < 1.05)].dropna(subset='soh'), x='odometer', y='soh', color='vin')